# IMPORTING NECESSARY LIBRARIES 

In [1]:
!pip install -U diffusers transformers accelerate scipy ftfy safetensors huggingface_hub
!pip install groq
!pip install requests
!pip install -U "peft>0.15"
!pip install qrcode




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 100.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 54.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

# BASE IMAGE GENERATION USING (SDXL)

In [ ]:
import os
import gc
import torch
from groq import Groq
from diffusers import DiffusionPipeline
from huggingface_hub import HfFolder, hf_hub_download
from transformers import CLIPTokenizerFast
from PIL import Image
import json
import time
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# === CONFIG ===
os.environ['GROQ_API_KEY'] = 'YOUR_GROQ_API_KEY_HERE'
client = Groq()

hf_token = "YOUR_HF_TOKEN"
HfFolder.save_token(hf_token)

# === Retry Session for Hugging Face Downloads ===
session = requests.Session()
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

# === Load CLIP Tokenizer with Retry ===
def load_clip_tokenizer():
    max_retries = 3
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries} to load CLIP tokenizer...")
            return CLIPTokenizerFast.from_pretrained(
                "openai/clip-vit-large-patch14",
                cache_dir="/kaggle/working/cache",
                local_files_only=False,
                token=hf_token,
                timeout=20  # Increased timeout
            )
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error loading tokenizer (attempt {attempt + 1}): {e}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)  # Exponential backoff
    return None

clip_tokenizer = load_clip_tokenizer()

# === SYSTEM PROMPT ===
system_prompt = '''
You are a world-class prompt engineer for Stable Diffusion. Your task is to rewrite user input into a single visual-only image generation prompt, strictly under 77 Stable Diffusion tokens (CLIP tokens), with no instructions or extra text.

🧠 Constraints:
- Use rich visual language: describe subject, composition, colors, lighting, mood, and style
- Ensure clean background or negative space with multiple reserved zones for text (e.g., top center, bottom center, left middle), keeping them clear
- NEVER include: any text, watermark, UI elements, logos
- NEVER return "Here is your prompt" or any commentary — only the prompt
- Must be one single line, no line breaks or markdown
- Use close to 70–77 CLIP tokens for maximum detail, not less

🎯 Visual Goal:
Create a poster-friendly EdTech scene — like a futuristic classroom with VR or AI elements, photorealistic, minimal clutter, and elegant.
'''

# === RAW PROMPT ===
raw_prompt = """
A cheerful young boy holding a laptop, the laptop screen showing futuristic robotics and a humanoid robot design. The boy is wearing a white shirt and blue jeans, smiling confidently. Background is colorful and tech-inspired with abstract shapes, light purple and orange tones, evoking a theme of kids' education in technology and coding. Professional lighting, realistic style.
"""

# === LLaMA 3 PROMPT ENHANCER ===
def enhance_prompt(user_prompt):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries} to enhance prompt...")
            response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.6,
                max_tokens=250,
                timeout=20
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ Error enhancing prompt (attempt {attempt + 1}): {e}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)
    return ""

# === SDXL PROMPT TRIMMER ===
def truncate_to_clip_limit(prompt):
    tokens = clip_tokenizer(prompt)["input_ids"]
    if len(tokens) > 77:
        trimmed = clip_tokenizer.decode(tokens[:77], skip_special_tokens=True).strip()
        print(f"⚠️ Prompt clipped to 77 tokens (original: {len(tokens)}).")
        return trimmed
    else:
        print(f"✅ Prompt within 77-token limit.")
        return prompt.strip()

# === LOAD SDXL MODEL ===
print("🔄 Loading Stable Diffusion XL...")
pipe = DiffusionPipeline.from_pretrained(
    "/kaggle/input/stable-diffusion-xl/pytorch/base-1-0/1",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)
pipe.to("cuda")
print("✅ SDXL loaded!")

# === PROMPT ENHANCEMENT AND GENERATION ===
print("🤖 Enhancing prompt...")
enhanced_prompt = enhance_prompt(raw_prompt)
final_prompt = truncate_to_clip_limit(enhanced_prompt)
print(f"📝 Final Prompt ({len(clip_tokenizer(final_prompt)['input_ids'])} tokens): {final_prompt}")

print("🎨 Generating 1024x1024 base image with SDXL...")
negative_prompt = (
    "blurry, deformed, extra limbs, low quality, distorted, watermark, text, cluttered, "
    "chaotic, bad anatomy, ugly face, lowres background, poorly rendered face, poorly drawn hands, "
    "mutated body parts, oversaturated, bad composition, disfigured, images cut out at the top, left, right, bottom, "
    "long legs, unnatural proportions, disproportionate body, malformed legs"
)

image = pipe(
    prompt=final_prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=25,
    guidance_scale=6.5,
    height=1024,
    width=1024,
    generator=torch.manual_seed(42)
).images[0]

image_path_1024 = "/kaggle/working/edtech_1024.png"
image.save(image_path_1024)
print(f"✅ 1024x1024 base image saved: {image_path_1024}")

# === CLEANUP ===
print("\n🧹 Cleaning up GPU memory...")
del pipe
torch.cuda.empty_cache()
gc.collect()
print("✅ All done!")

🔄 Attempt 1/3 to load CLIP tokenizer...


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

🔄 Loading Stable Diffusion XL...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ SDXL loaded!
🤖 Enhancing prompt...
🔄 Attempt 1/3 to enhance prompt...
✅ Prompt within 77-token limit.
📝 Final Prompt (55 tokens): A bright, confident boy in a white shirt and blue jeans holds a laptop displaying futuristic robotics, surrounded by abstract shapes in light purple and orange hues, amidst a clean, professional, photorealistic futuristic classroom with subtle neon accents, soft focus, and elegant atmospheric lighting.
🎨 Generating 1024x1024 base image with SDXL...


  0%|          | 0/25 [00:00<?, ?it/s]

✅ 1024x1024 base image saved: /kaggle/working/edtech_1024.png

🧹 Cleaning up GPU memory...
✅ All done!


# ARCHITECTURE SELECTION FOR BASE IMAGE(Llama 4)

In [ ]:
import os
import base64
import json
from PIL import Image, ImageDraw
from groq import Groq

# === CONFIGURATION ===
API_KEY = "YOUR_GROQ_API_KEY"
TARGET_SIZE = 1024
base_image_path = "/kaggle/working/edtech_1024.png"
output_path = "visualization_chosen.png"

# === IMAGE ENCODING ===
def encode_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

base_image_b64 = encode_image_base64(base_image_path)

# === ARCHITECTURE PROMPTS ===
asymmetric_prompt = """
You are an EXPERT EdTech Marketing Designer creating a visually striking 1024x1024 poster for a Generative AI Bootcamp, targeting a Gen Z audience.

CRITICAL REQUIREMENTS:
- Generate EXACTLY 6 text zones with NO overlaps.
- Minimum 120px distance between zone centers.
- Minimum 40px buffer around each bounding box.
- All zones must fit within the 1024x1024 canvas with 30px padding.
- Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
  - hero_headline: 700x180 max, font_size: 70px, center-aligned
  - hero_subline: 600x120 max, font_size: 45px, center-aligned
  - benefit_statement: 350x250 max, font_size: 36px
  - testimonial: 350x200 max, font_size: 34px
  - success_metrics: 400x150 max, font_size: 42px
  - cta_primary: 400x80 max, font_size: 55px
- Let the model decide the best box positions for aesthetics and negative space—do NOT use a grid or fixed columns/rows.
- Specify 'font_family', 'font_size', and 'text_align' in JSON output.

ZONE TYPES (must include all 6):
1. hero_headline
2. hero_subline
3. benefit_statement
4. testimonial
5. success_metrics
6. cta_primary

REQUIRED JSON FORMAT:
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 6,
    "layout_strategy": "asymmetric_freeform"
  },
  "text_regions": [
    {
      "type": "<zone_type>",
      "x": <int>,
      "y": <int>,
      "width": <int>,
      "height": <int>,
      "font_family": "<font_name>",
      "font_size": <int>,
      "text_align": "<alignment>"
    },
    ...
  ]
}

Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
"""

adaptive_prompt = """
You are an EXPERT EdTech Marketing Designer creating a visually striking 1024x1024 poster for a Generative AI Bootcamp, targeting a Gen Z audience.

CRITICAL REQUIREMENTS:
- Generate EXACTLY 6 text zones with NO overlaps beyond 15px.
- Minimum 80px gap between zone edges.
- All zones must fit within the 1024x1024 canvas with 20px padding.
- Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
  - hero_headline: 600x160 max, font_size: 60px, center-aligned
  - hero_subline: 500x100 max, font_size: 40px, center-aligned
  - benefit_statement: 350x220 max, font_size: 35px
  - testimonial: 350x180 max, font_size: 32px
  - success_metrics: 400x150 max, font_size: 40px
  - cta_primary: 350x70 max, font_size: 45px
- Arrange zones dynamically, filling available space without a grid, adapting to the image's negative areas.
- Specify 'font_family' (Inter or Poppins), 'font_size', and 'text_align' in JSON output.

ZONE TYPES (must include all 6):
1. hero_headline
2. hero_subline
3. benefit_statement
4. testimonial
5. success_metrics
6. cta_primary

REQUIRED JSON FORMAT:
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 6,
    "layout_strategy": "adaptive_grid_free"
  },
  "text_regions": [
    {
      "type": "<zone_type>",
      "x": <int>,
      "y": <int>,
      "width": <int>,
      "height": <int>,
      "font_family": "<font_name>",
      "font_size": <int>,
      "text_align": "<alignment>"
    },
    ...
  ]
}

Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
"""

staggered_prompt = """
You are an EXPERT EdTech Marketing Designer creating a visually striking 1024x1024 poster for a Generative AI Bootcamp, targeting a Gen Z audience.

CRITICAL REQUIREMENTS:
- Generate EXACTLY 5 text zones with NO overlaps beyond 10px horizontally.
- Minimum 90px vertical gap between tiers (top, middle, bottom).
- All zones must fit within the 1024x1024 canvas with 25px padding.
- Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
  - hero_headline: 500x140 max, font_size: 65px, center-aligned
  - hero_subline: 400x80 max, font_size: 38px, center-aligned
  - benefit_statement: 300x180 max, font_size: 32px, left-aligned
  - testimonial: 300x150 max, font_size: 30px, right-aligned
  - cta_primary: 350x60 max, font_size: 48px, center-aligned
- Arrange zones in a staggered horizon pattern: hero_headline and hero_subline on top tier (left/right), benefit_statement and testimonial on middle tier (left/right), cta_primary on bottom tier (center).
- Specify 'font_family' (Montserrat or Open Sans), 'font_size', and 'text_align' in JSON output.

ZONE TYPES (must include all 5):
1. hero_headline
2. hero_subline
3. benefit_statement
4. testimonial
5. cta_primary

REQUIRED JSON FORMAT:
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 5,
    "layout_strategy": "staggered_horizon"
  },
  "text_regions": [
    {
      "type": "<zone_type>",
      "x": <int>,
      "y": <int>,
      "width": <int>,
      "height": <int>,
      "font_family": "<font_name>",
      "font_size": <int>,
      "text_align": "<alignment>"
    },
    ...
  ]
}

Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
"""

# === LLAMA 4 MAVERICK CALL ===
def call_llama4_maverick(prompt, image_b64):
    client = Groq(api_key=API_KEY)
    print("🔄 Sending prompt and image to Llama 4 Maverick...")
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                ]
            }
        ],
        max_tokens=2048,
        temperature=0.7,
        response_format={"type": "json_object"},
    )
    print("✅ Received response from Maverick.")
    return completion.choices[0].message.content

# === DECISION MAKING ===
def choose_best_layout(image_b64):
    decision_prompt = f"""
    You are an EXPERT EdTech Marketing Designer. Analyze the provided 1024x1024 base image for a Generative AI Bootcamp poster targeting a Gen Z audience. Based on the image's layout, negative space, and aesthetic, select the optimal architecture from the following options:
    1. Asymmetric Freeform: Best for irregular, creative layouts with lots of negative space (6 zones, 120px center distance, 40px buffer).
    2. Adaptive Grid-Free: Ideal for filling space dynamically with balanced distribution (6 zones, 80px edge gap, 15px max overlap).
    3. Staggered Horizon: Suits tiered, structured designs with vertical flow (5 zones, 90px vertical gap, 10px horizontal overlap).

    Return ONLY the chosen layout strategy as a string: "asymmetric_freeform", "adaptive_grid_free", or "staggered_horizon". Do NOT include any commentary, explanation, or extra text.
    """
    client = Groq(api_key=API_KEY)
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": decision_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                ]
            }
        ],
        max_tokens=10,
        temperature=0.5,
        response_format={"type": "text"},
    )
    return completion.choices[0].message.content.strip()

# === VALIDATION FUNCTIONS ===
def validate_asymmetric_layout(text_regions):
    errors = []
    max_sizes = {
        "hero_headline": (700, 180, 70),
        "hero_subline": (600, 120, 45),
        "benefit_statement": (350, 250, 36),
        "testimonial": (350, 200, 34),
        "success_metrics": (400, 150, 42),
        "cta_primary": (400, 80, 55)
    }
    for i, reg1 in enumerate(text_regions):
        t = reg1["type"]
        max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
        if reg1["width"] > max_w or reg1["height"] > max_h:
            errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
        if reg1.get("font_size", 0) > max_font:
            errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
        for j, reg2 in enumerate(text_regions):
            if i < j:
                if check_overlap(reg1, reg2, buffer=40):
                    errors.append(f"Overlap between {reg1['type']} and {reg2['type']}")
                cx1, cy1 = reg1["x"] + reg1["width"]//2, reg1["y"] + reg1["height"]//2
                cx2, cy2 = reg2["x"] + reg2["width"]//2, reg2["y"] + reg2["height"]//2
                dist = ((cx1-cx2)**2 + (cy1-cy2)**2)**0.5
                if dist < 120:
                    errors.append(f"Center too close: {reg1['type']} and {reg2['type']} ({dist:.1f}px)")
        if reg1["x"] < 30 or reg1["y"] < 30 or reg1["x"]+reg1["width"] > TARGET_SIZE-30 or reg1["y"]+reg1["height"] > TARGET_SIZE-30:
            errors.append(f"{t} outside canvas padding")
    return errors

def validate_adaptive_layout(text_regions):
    errors = []
    max_sizes = {
        "hero_headline": (600, 160, 60),
        "hero_subline": (500, 100, 40),
        "benefit_statement": (350, 220, 35),
        "testimonial": (350, 180, 32),
        "success_metrics": (400, 150, 40),
        "cta_primary": (350, 70, 45)
    }
    for i, reg1 in enumerate(text_regions):
        t = reg1["type"]
        max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
        if reg1["width"] > max_w or reg1["height"] > max_h:
            errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
        if reg1.get("font_size", 0) > max_font:
            errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
        for j, reg2 in enumerate(text_regions):
            if i < j:
                if check_adaptive_overlap(reg1, reg2):
                    errors.append(f"Overlap or insufficient gap between {reg1['type']} and {reg2['type']}")
        if reg1["x"] < 20 or reg1["y"] < 20 or reg1["x"]+reg1["width"] > TARGET_SIZE-20 or reg1["y"]+reg1["height"] > TARGET_SIZE-20:
            errors.append(f"{t} outside canvas padding")
    return errors

def validate_staggered_layout(text_regions):
    errors = []
    max_sizes = {
        "hero_headline": (500, 140, 65),
        "hero_subline": (400, 80, 38),
        "benefit_statement": (300, 180, 32),
        "testimonial": (300, 150, 30),
        "cta_primary": (350, 60, 48)
    }
    for i, reg1 in enumerate(text_regions):
        t = reg1["type"]
        max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
        if reg1["width"] > max_w or reg1["height"] > max_h:
            errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
        if reg1.get("font_size", 0) > max_font:
            errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
        for j, reg2 in enumerate(text_regions):
            if i < j:
                if check_staggered_overlap(reg1, reg2):
                    errors.append(f"Overlap or insufficient gap between {reg1['type']} and {reg2['type']}")
        if reg1["x"] < 25 or reg1["y"] < 25 or reg1["x"]+reg1["width"] > TARGET_SIZE-25 or reg1["y"]+reg1["height"] > TARGET_SIZE-25:
            errors.append(f"{t} outside canvas padding")
    return errors

# === UTILITY FUNCTIONS ===
def check_overlap(region1, region2, buffer=40):
    x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
    x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
    x1b, y1b, w1b, h1b = x1-buffer, y1-buffer, w1+2*buffer, h1+2*buffer
    x2b, y2b, w2b, h2b = x2-buffer, y2-buffer, w2+2*buffer, h2+2*buffer
    return not (x1b + w1b < x2b or x2b + w2b < x1b or y1b + h1b < y2b or y2b + h2b < y1b)

def check_adaptive_overlap(region1, region2, min_gap=80, max_overlap=15):
    x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
    x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
    x1_right, y1_bottom = x1 + w1, y1 + h1
    x2_right, y2_bottom = x2 + w2, y2 + h2
    horizontal_gap = min(abs(x1 - x2_right), abs(x1_right - x2))
    vertical_gap = min(abs(y1 - y2_bottom), abs(y1_bottom - y2))
    overlap_x = max(0, min(x1_right, x2_right) - max(x1, x2))
    overlap_y = max(0, min(y1_bottom, y2_bottom) - max(y1, y2))
    total_overlap = overlap_x * overlap_y
    if horizontal_gap < min_gap or vertical_gap < min_gap:
        return True
    if total_overlap > max_overlap:
        return True
    return False

def check_staggered_overlap(region1, region2, min_vertical_gap=90, max_horizontal_overlap=10):
    x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
    x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
    x1_right, y1_bottom = x1 + w1, y1 + h1
    x2_right, y2_bottom = x2 + w2, y2 + h2
    vertical_gap = min(abs(y1 - y2_bottom), abs(y1_bottom - y2))
    if vertical_gap < min_vertical_gap and region1["type"] != region2["type"]:
        return True
    overlap_x = max(0, min(x1_right, x2_right) - max(x1, x2))
    if overlap_x > max_horizontal_overlap:
        return True
    return False

# === VISUALIZATION ===
def visualize_layout(text_regions, base_image_path, filename="visualization_chosen.png"):
    img = Image.open(base_image_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    colors = ["#00CED1", "#FFD700", "#FF69B4", "#ADFF2F", "#FF6347", "#1E90FF", "#20B2AA"]
    for idx, reg in enumerate(text_regions):
        color = colors[idx % len(colors)]
        x, y, w, h = reg["x"], reg["y"], reg["width"], reg["height"]
        label = f"#{idx+1} {reg['type']} {reg.get('font_size','')}px"
        draw.rectangle([x, y, x+w, y+h], outline=color, width=4)
        draw.text((x+8, y+8), label, fill=color)
    img.save(filename)
    print(f"🖼️ Chosen layout visualization saved as {filename}")

# === MAIN WORKFLOW ===
if __name__ == "__main__":
    # Decide the best layout
    chosen_layout = choose_best_layout(base_image_b64)
    print(f"🔍 Chosen layout: {chosen_layout}")

    # Run the corresponding prompt
    if chosen_layout == "asymmetric_freeform":
        json_str = call_llama4_maverick(asymmetric_prompt, base_image_b64)
        validation_func = validate_asymmetric_layout
    elif chosen_layout == "adaptive_grid_free":
        json_str = call_llama4_maverick(adaptive_prompt, base_image_b64)
        validation_func = validate_adaptive_layout
    elif chosen_layout == "staggered_horizon":
        json_str = call_llama4_maverick(staggered_prompt, base_image_b64)
        validation_func = validate_staggered_layout
    else:
        raise ValueError("Invalid layout strategy selected.")

    print("🔎 Parsing JSON...")
    layout_data = json.loads(json_str)
    print(json.dumps(layout_data, indent=2))

    print("🔎 Validating layout...")
    validation_errors = validation_func(layout_data["text_regions"])
    if validation_errors:
        print("❗ Validation errors found:")
        for err in validation_errors:
            print("  -", err)
    else:
        print("✅ Layout passes all constraints.")

    visualize_layout(layout_data["text_regions"], base_image_path, output_path)

    print("\n--- DEBUG INFO ---")
    print(json.dumps(layout_data, indent=2))
    if validation_errors:
        print("\n❗ Please review the above errors and consider re-prompting Maverick or adjusting constraints.")
    else:
        print("\n🎉 Chosen layout is ready for text insertion and further design steps.")

🔍 Chosen layout: asymmetric_freeform
🔄 Sending prompt and image to Llama 4 Maverick...
✅ Received response from Maverick.
🔎 Parsing JSON...
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 6,
    "layout_strategy": "asymmetric_freeform"
  },
  "text_regions": [
    {
      "type": "hero_headline",
      "x": 512,
      "y": 120,
      "width": 600,
      "height": 120,
      "font_family": "Montserrat",
      "font_size": 60,
      "text_align": "center"
    },
    {
      "type": "hero_subline",
      "x": 512,
      "y": 280,
      "width": 500,
      "height": 100,
      "font_family": "Lato",
      "font_size": 40,
      "text_align": "center"
    },
    {
      "type": "benefit_statement",
      "x": 200,
      "y": 500,
      "width": 300,
      "height": 200,
      "font_family": "Open Sans",
      "font_size": 32,
      "text_align": "left"
    },
    {
      "type": "testimonial",
      "x": 700,
      "y": 450,
      "width": 250,
      "height": 15

# SAVING THE OP AS JSON

In [20]:
import json

# Save the JSON layout to a file
with open("layout.json", "w") as f:
    json.dump(layout_data, f, indent=2)
print("💾 Saved layout JSON as layout.json")


💾 Saved layout JSON as layout.json


# UNIVERSAL PIL CODE 

In [21]:
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import json
import os
import qrcode

# --- CONFIGURATION ---
FONT_BASE_PATH = "/kaggle/input/fontsss"
TARGET_SIZE = 1024
CLEAN_OUTPUT_PATH = "/kaggle/working/final_poster_clean.png"
DEBUG_OUTPUT_PATH = "/kaggle/working/visualization_maverick.png"
IMAGE_PATH = "/kaggle/working/edtech_1024.png"

# Logo paths
LOGO_PATHS = {
    "linkedin": "/kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png",
    "twitter": "/kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png"
}

# Font paths for all architectures
font_paths = {
    "montserrat-bold": "/kaggle/input/data2/archive (4)/Montserrat/static/Montserrat-Bold.ttf",
    "open-sans-regular": "/kaggle/input/data2/archive (4)/Open_Sans/static/OpenSans-Regular.ttf",
    "poppins-bold": "/kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf",
    "inter-regular": "/kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf"
}

# Content mapping for each zone type
ZONE_CONTENT = {
    "hero_headline": "Master Gen AI in 12 Weeks",
    "hero_subline": "Build Tomorrow's Tech Skills Today",
    "benefit_statement": "• Build killer portfolios\n• Guaranteed job interviews",
    "testimonial": "\"This bootcamp changed my career trajectory completely!\" - Sarah K.",
    "success_metrics": "95% Job Placement\n$75K+ Average Salary",
    "cta_primary": "ENROLL NOW"
}

# --- UTILITY FUNCTIONS ---
def get_font_with_fallback(zone_type, font_size, font_family=None):
    print(f"\n🔍 Loading font for {zone_type} with size {font_size}px, family: {font_family}")
    
    # Default font mapping based on zone type
    if not font_family:
        if zone_type in ["hero_headline", "success_metrics", "cta_primary"]:
            font_family = "montserrat-bold"  # Default bold for headlines/CTAs
        else:
            font_family = "open-sans-regular"  # Default regular for others
    
    # Map font_family from JSON to available fonts
    font_map = {
        "Montserrat": "montserrat-bold",
        "Open Sans": "open-sans-regular",
        "Poppins": "poppins-bold",
        "Inter": "inter-regular"
    }
    font_key = font_map.get(font_family.split()[0], "montserrat-bold")  # Fallback to Montserrat
    
    font_path = font_paths.get(font_key)
    font_variant = font_key.replace("-", " ").title()
    
    if font_path and os.path.exists(font_path):
        try:
            print(f"   ✅ Loading {font_variant} from {font_path}")
            return ImageFont.truetype(font_path, font_size)
        except Exception as e:
            print(f"   ❌ Error loading {font_path}: {str(e)}")
    print("   ⚠️ Font loading failed, falling back to default PIL font")
    return ImageFont.load_default()

def hex_to_rgba(hex_color):
    try:
        hex_color = hex_color.lstrip("#")
        if len(hex_color) == 6:
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (255,)
        return (255, 255, 255, 255)
    except Exception as e:
        print(f"⚠️ Error converting hex color {hex_color}: {e}")
        return (255, 255, 255, 255)

def add_text_with_shadow(draw, position, text, font, text_color):
    x, y = position
    shadow_offset = 2
    draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=(0, 0, 0, 120))  # Subtle shadow
    draw.text((x, y), text, font=font, fill=text_color)  # Main text

def create_qr_code(draw, image, x, y, width, height):
    if width <= 0 or height <= 0:
        print(f"⚠️ Invalid QR dimensions: width={width}, height={height}, using defaults")
        width, height = 100, 100
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=1
    )
    qr.add_data("https://genai-bootcamp.com")
    qr.make(fit=True)
    qr_image = qr.make_image(fill_color="black", back_color="white").convert("RGBA")
    qr_image = qr_image.resize((width, height), Image.Resampling.LANCZOS)
    enhancer = ImageEnhance.Brightness(qr_image)
    qr_image = enhancer.enhance(0.8)
    image.paste(qr_image, (int(x), int(y)), qr_image)
    print(f"✅ Drew QR code at ({x}, {y}) with size {width}x{height}")

def draw_social_logos(draw, image, x, y, max_width, max_height):
    try:
        linkedin_logo = Image.open(LOGO_PATHS["linkedin"]).convert("RGBA")
        twitter_logo = Image.open(LOGO_PATHS["twitter"]).convert("RGBA")
        max_logo_size = min(40, max_height - 10)
        linkedin_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        twitter_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        linkedin_x = x
        linkedin_y = y + (max_height - linkedin_logo.height) // 2
        twitter_x = linkedin_x + linkedin_logo.width + 15
        twitter_y = y + (max_height - twitter_logo.height) // 2
        if twitter_x + twitter_logo.width > x + max_width:
            twitter_x = x + max_width - twitter_logo.width
            linkedin_x = twitter_x - linkedin_logo.width - 15
        enhancer = ImageEnhance.Brightness
        linkedin_logo = enhancer(linkedin_logo).enhance(0.7)
        twitter_logo = enhancer(twitter_logo).enhance(0.7)
        image.paste(linkedin_logo, (int(linkedin_x), int(linkedin_y)), linkedin_logo)
        image.paste(twitter_logo, (int(twitter_x), int(twitter_y)), twitter_logo)
        print(f"✅ Drew logos: LinkedIn at ({linkedin_x}, {linkedin_y}), Twitter at ({twitter_x}, {twitter_y})")
    except Exception as e:
        print(f"❌ Error drawing social logos: {e}")

def wrap_text(draw, text, font, max_width, max_height, font_size, zone_type):
    lines = []
    adjusted_font = font
    adjusted_font_size = font_size
    print(f"📏 Wrapping text for {zone_type}: '{text}'")
    print(f"    Max width: {max_width}px, Max height: {max_height}px, Initial font_size: {font_size}px")
    min_font_size = max(12, int(font_size * 0.6))
    if "•" in text:
        raw_lines = text.split("\n")
    else:
        raw_lines = [text]
    while adjusted_font_size >= min_font_size:
        current_lines = []
        for line in raw_lines:
            if not line.strip():
                continue
            if line.strip().startswith("•"):
                bullet_text = line.strip()
                text_bbox = draw.textbbox((0, 0), bullet_text, font=adjusted_font)
                text_width = text_bbox[2] - text_bbox[0]
                if text_width <= max_width - 20:
                    current_lines.append(bullet_text)
                else:
                    words = bullet_text[2:].split()
                    current_line = ["•"]
                    for word in words:
                        test_line = " ".join(current_line + [word])
                        text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                        text_width = text_bbox[2] - text_bbox[0]
                        if text_width <= max_width - 20:
                            current_line.append(word)
                        else:
                            if len(current_line) > 1:
                                current_lines.append(" ".join(current_line))
                            current_line = ["•", word]
                    if len(current_line) > 1:
                        current_lines.append(" ".join(current_line))
            else:
                words = line.split()
                current_line = []
                for word in words:
                    test_line = " ".join(current_line + [word])
                    text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                    text_width = text_bbox[2] - text_bbox[0]
                    if text_width <= max_width:
                        current_line.append(word)
                    else:
                        if current_line:
                            current_lines.append(" ".join(current_line))
                        current_line = [word]
                if current_line:
                    current_lines.append(" ".join(current_line))
        if current_lines:
            text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
            line_height = text_bbox[3] - text_bbox[1] + 6
            total_height = len(current_lines) * line_height
            print(f"    Font size {adjusted_font_size}px: {len(current_lines)} lines, Total height: {total_height}px")
            if total_height <= max_height or adjusted_font_size == min_font_size:
                lines = current_lines
                break
        adjusted_font_size -= 1
        adjusted_font = get_font_with_fallback(zone_type, adjusted_font_size)
    if not lines and text.strip():
        print("⚠️ Text doesn't fit, forcing at least one line")
        lines = [text[:30] + "..." if len(text) > 30 else text]
    print(f"    Final: {len(lines)} lines with font size {adjusted_font_size}px")
    return lines, adjusted_font, adjusted_font_size

def get_text_color_for_zone(zone_type):
    color_map = {
        "hero_headline": "#FFFFFF",
        "hero_subline": "#E0E0E0",
        "benefit_statement": "#D3D3D3",
        "testimonial": "#D3D3D3",
        "success_metrics": "#E0E0E0",
        "cta_primary": "#00CED1"
    }
    return color_map.get(zone_type, "#FFFFFF")

def get_text_alignment(zone_type, text_align):
    if zone_type in ["hero_headline", "hero_subline", "cta_primary"]:
        return "center"
    return text_align if text_align else "left"

def draw_zone_content(draw, image, zone_region, show_debug=False):
    zone_type = zone_region.get("type", "")
    x = zone_region.get("x", 0)
    y = zone_region.get("y", 0)
    width = zone_region.get("width", 300)
    height = zone_region.get("height", 100)
    font_size = zone_region.get("font_size", 24)
    text_align = zone_region.get("text_align", "left")
    font_family = zone_region.get("font_family", None)
    
    text = ZONE_CONTENT.get(zone_type, f"Sample {zone_type} text")
    text_color = hex_to_rgba(get_text_color_for_zone(zone_type))
    alignment = get_text_alignment(zone_type, text_align)
    
    print(f"\n🎨 Drawing {zone_type}: '{text}' at ({x}, {y}) size {width}x{height}")
    
    image_w, image_h = image.size
    margin = 30
    x = max(margin, min(x, image_w - width - margin))
    y = max(margin, min(y, image_h - height - margin))
    
    font = get_font_with_fallback(zone_type, font_size, font_family)
    
    lines, adjusted_font, adjusted_font_size = wrap_text(
        draw, text, font, width - 20, height - 20, font_size, zone_type
    )
    
    if not lines:
        print("⚠️ No text to draw")
        return
    
    text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
    line_height = text_bbox[3] - text_bbox[1] + 6
    total_text_height = len(lines) * line_height
    if alignment == "center":
        start_y = y + (height - total_text_height) // 2
    else:
        start_y = y + 10
    
    for i, line in enumerate(lines):
        line_y = start_y + i * line_height
        if alignment == "center":
            text_bbox = draw.textbbox((0, 0), line, font=adjusted_font)
            text_width = text_bbox[2] - text_bbox[0]
            line_x = x + (width - text_width) // 2
        else:
            line_x = x + 10
        add_text_with_shadow(draw, (line_x, line_y), line, adjusted_font, text_color)
    
    print(f"✅ Drew {zone_type} with {len(lines)} lines (font size: {adjusted_font_size}px)")
    
    if show_debug:
        debug_colors = {
            'hero_headline': '#00FFFF', 'hero_subline': '#FFFF00', 'benefit_statement': '#00FF00',
            'testimonial': '#FF00FF', 'success_metrics': '#0000FF', 'cta_primary': '#FFA500'
        }
        color = debug_colors.get(zone_type, '#FF0000')
        draw.rectangle([x, y, x + width, y + height], outline=color, width=2)
        draw.text((x + 5, y + 5), f"{zone_type}", fill=color, font=get_font_with_fallback("default", 12))

def render_poster():
    print("🎨 Llama 4 Maverick Universal Poster Renderer")
    print("=" * 50)
    
    json_path = "/kaggle/working/layout.json"
    if not os.path.exists(json_path):
        print(f"❌ JSON file not found: {json_path}")
        return None, None
    
    print(f"📄 Loading JSON from: {json_path}")
    try:
        with open(json_path, "r") as f:
            layout_data = json.load(f)
        text_regions = layout_data.get("text_regions", [])
        print(f"📊 Found {len(text_regions)} text regions")
    except Exception as e:
        print(f"❌ Error loading JSON: {e}")
        return None, None
    
    if not text_regions:
        print("❌ No text regions found in JSON!")
        return None, None
    
    print(f"🖼️ Loading base image: {IMAGE_PATH}")
    try:
        base_image = Image.open(IMAGE_PATH).convert("RGBA")
        image_w, image_h = base_image.size
        print(f"✅ Image loaded: {image_w}x{image_h}")
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None
    
    clean_image = base_image.copy()
    debug_image = base_image.copy()
    clean_draw = ImageDraw.Draw(clean_image)
    debug_draw = ImageDraw.Draw(debug_image)
    
    successful_zones = 0
    for i, region in enumerate(text_regions, 1):
        zone_type = region.get("type", f"zone_{i}")
        print(f"\n--- Processing Region {i}/{len(text_regions)}: {zone_type} ---")
        try:
            draw_zone_content(clean_draw, clean_image, region, show_debug=False)
            draw_zone_content(debug_draw, debug_image, region, show_debug=True)
            successful_zones += 1
        except Exception as e:
            print(f"❌ Error rendering {zone_type}: {e}")
            continue
    
    print("\n🎯 Adding hardcoded QR code...")
    create_qr_code(clean_draw, clean_image, 900, 880, 100, 100)
    create_qr_code(debug_draw, debug_image, 900, 880, 100, 100)
    debug_draw.rectangle([900, 880, 1000, 980], outline='#808080', width=2)
    debug_draw.text((905, 885), "QR Code", fill='#808080', font=get_font_with_fallback("default", 12))
    
    print("\n🎯 Adding hardcoded social logos...")
    draw_social_logos(clean_draw, clean_image, 30, 880, 120, 60)
    draw_social_logos(debug_draw, debug_image, 30, 880, 120, 60)
    debug_draw.rectangle([30, 880, 150, 940], outline='#4169E1', width=2)
    debug_draw.text((35, 885), "Social Logos", fill='#4169E1', font=get_font_with_fallback("default", 12))
    
    print(f"\n📊 Successfully rendered {successful_zones} text zones + QR + Logos")
    
    clean_path = None
    debug_path = None
    try:
        clean_image.save(CLEAN_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Clean poster saved: {CLEAN_OUTPUT_PATH}")
        clean_path = CLEAN_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving clean image: {e}")
    try:
        debug_image.save(DEBUG_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Debug poster saved: {DEBUG_OUTPUT_PATH}")
        debug_path = DEBUG_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving debug image: {e}")
    
    return clean_path, debug_path

if __name__ == "__main__":
    print("🚀 Starting universal poster generation from /kaggle/working/layout.json...")
    print("\n🔍 Checking required files:")
    for name, path in font_paths.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Font {name}: {path}")
    for name, path in LOGO_PATHS.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Logo {name}: {path}")
    json_path = "/kaggle/working/layout.json"
    json_exists = os.path.exists(json_path)
    status = "✅" if json_exists else "❌"
    print(f"   {status} JSON file: {json_path}")
    clean_result, debug_result = render_poster()
    if clean_result and debug_result:
        print(f"\n🎉 SUCCESS! Generated two versions:")
        print(f"   📸 CLEAN POSTER: {clean_result}")
        print(f"   🔧 DEBUG VERSION: {debug_result}")
    else:
        print("\n❌ Failed to create poster. Check error messages above.")
    print("\n🏁 Poster generation completed!")

🚀 Starting universal poster generation from /kaggle/working/layout.json...

🔍 Checking required files:
   ✅ Font montserrat-bold: /kaggle/input/data2/archive (4)/Montserrat/static/Montserrat-Bold.ttf
   ✅ Font open-sans-regular: /kaggle/input/data2/archive (4)/Open_Sans/static/OpenSans-Regular.ttf
   ✅ Font poppins-bold: /kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf
   ✅ Font inter-regular: /kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf
   ✅ Logo linkedin: /kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png
   ✅ Logo twitter: /kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png
   ✅ JSON file: /kaggle/working/layout.json
🎨 Llama 4 Maverick Universal Poster Renderer
📄 Loading JSON from: /kaggle/working/layout.json
📊 Found 6 text regions
🖼️ Loading base image: /kaggle/working/edtech_1024.png
✅ Image loaded: 1024x1024

--- Processing Region 1/6: hero_headline ---

🎨 Drawing hero_headline: 'Master Gen AI in 12 Weeks' at (512, 120) size 60

# TESTTING###########

# ARCHITECURES(To be Fused)

# Assymetic Architecture


In [ ]:
import os
import base64
import json
from PIL import Image, ImageDraw
from groq import Groq

# === CONFIGURATION ===
API_KEY = "YOUR_GROQ_API_KEY_HERE"
TARGET_SIZE = 1024
layout_name = "asymmetric_freeform"
base_image_path = "/kaggle/working/edtech_1024.png"
output_path = "visualization.png"
raw_prompt = """
Diverse young adults in a modern tech classroom, focused on laptops, bright daylight, futuristic lighting, digital whiteboards, soft depth of field, cinematic shot, tech startup vibes, motivational ambiance, 4K photo-realistic
"""

# === IMAGE ENCODING ===
def encode_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

base_image_b64 = encode_image_base64(base_image_path)

# === LOOSE, GEMINI-BOX PROMPT ===
base_prompt = f"""
You are an EXPERT EdTech Marketing Designer creating a visually striking {TARGET_SIZE}x{TARGET_SIZE} poster for a Generative AI Bootcamp, targeting a Gen Z audience.

CRITICAL REQUIREMENTS:
- Generate EXACTLY 6 text zones with NO overlaps.
- Minimum 120px distance between zone centers.
- Minimum 40px buffer around each bounding box.
- All zones must fit within the {TARGET_SIZE}x{TARGET_SIZE} canvas with 30px padding.
- Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
  - hero_headline: 700x180 max, font_size: 70px, center-aligned
  - hero_subline: 600x120 max, font_size: 45px, center-aligned
  - benefit_statement: 350x250 max, font_size: 36px
  - testimonial: 350x200 max, font_size: 34px
  - success_metrics: 400x150 max, font_size: 42px
  - cta_primary: 400x80 max, font_size: 55px
- Let the model decide the best box positions for aesthetics and negative space—do NOT use a grid or fixed columns/rows.
- Specify 'font_family', 'font_size', and 'text_align' in JSON output.

ZONE TYPES (must include all 6):
1. hero_headline
2. hero_subline
3. benefit_statement
4. testimonial
5. success_metrics
6. cta_primary

REQUIRED JSON FORMAT:
{{
  "visual_analysis": {{
    "poster_size": "{TARGET_SIZE}x{TARGET_SIZE}",
    "total_zones": 6,
    "layout_strategy": "{layout_name}"
  }},
  "text_regions": [
    {{
      "type": "<zone_type>",
      "x": <int>,
      "y": <int>,
      "width": <int>,
      "height": <int>,
      "font_family": "<font_name>",
      "font_size": <int>,
      "text_align": "<alignment>"
    }},
    ...
  ]
}}

Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
"""

# === LLAMA 4 MAVERICK CALL ===
def call_llama4_maverick(prompt, image_b64):
    client = Groq(api_key=API_KEY)
    print("🔄 Sending prompt and image to Llama 4 Maverick...")
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                ]
            }
        ],
        max_tokens=2048,
        temperature=0.9,
        response_format={"type": "json_object"},
    )
    print("✅ Received response from Maverick.")
    return completion.choices[0].message.content

# === VALIDATION ===
def check_overlap(region1, region2, buffer=40):
    x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
    x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
    x1b, y1b, w1b, h1b = x1-buffer, y1-buffer, w1+2*buffer, h1+2*buffer
    x2b, y2b, w2b, h2b = x2-buffer, y2-buffer, w2+2*buffer, h2+2*buffer
    return not (x1b + w1b < x2b or x2b + w2b < x1b or y1b + h1b < y2b or y2b + h2b < y1b)

def validate_layout(text_regions):
    errors = []
    max_sizes = {
        "hero_headline": (700, 180, 70),
        "hero_subline": (600, 120, 45),
        "benefit_statement": (350, 250, 36),
        "testimonial": (350, 200, 34),
        "success_metrics": (400, 150, 42),
        "cta_primary": (400, 80, 55)
    }
    for i, reg1 in enumerate(text_regions):
        t = reg1["type"]
        max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
        if reg1["width"] > max_w or reg1["height"] > max_h:
            errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
        if reg1.get("font_size", 0) > max_font:
            errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
        for j, reg2 in enumerate(text_regions):
            if i < j:
                if check_overlap(reg1, reg2):
                    errors.append(f"Overlap between {reg1['type']} and {reg2['type']}")
                cx1, cy1 = reg1["x"] + reg1["width"]//2, reg1["y"] + reg1["height"]//2
                cx2, cy2 = reg2["x"] + reg2["width"]//2, reg2["y"] + reg2["height"]//2
                dist = ((cx1-cx2)**2 + (cy1-cy2)**2)**0.5
                if dist < 120:
                    errors.append(f"Center too close: {reg1['type']} and {reg2['type']} ({dist:.1f}px)")
        if reg1["x"] < 30 or reg1["y"] < 30 or reg1["x"]+reg1["width"] > TARGET_SIZE-30 or reg1["y"]+reg1["height"] > TARGET_SIZE-30:
            errors.append(f"{t} outside canvas padding")
    return errors

# === VISUALIZATION ===
def visualize_layout(text_regions, base_image_path, filename="visualization.png"):
    img = Image.open(base_image_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    colors = ["#00CED1", "#FFD700", "#FF69B4", "#ADFF2F", "#FF6347", "#1E90FF"]
    for idx, reg in enumerate(text_regions):
        color = colors[idx % len(colors)]
        x, y, w, h = reg["x"], reg["y"], reg["width"], reg["height"]
        label = f"#{idx+1} {reg['type']} {reg.get('font_size','')}px"
        draw.rectangle([x, y, x+w, y+h], outline=color, width=4)
        draw.text((x+8, y+8), label, fill=color)
    img.save(filename)
    print(f"🖼️ Layout visualization saved as {filename}")

# === MAIN WORKFLOW ===
if __name__ == "__main__":
    json_str = call_llama4_maverick(base_prompt, base_image_b64)
    print("🔎 Parsing JSON...")
    layout_data = json.loads(json_str)
    print(json.dumps(layout_data, indent=2))

    print("🔎 Validating layout...")
    validation_errors = validate_layout(layout_data["text_regions"])
    if validation_errors:
        print("❗ Validation errors found:")
        for err in validation_errors:
            print("  -", err)
    else:
        print("✅ Layout passes all constraints.")

    visualize_layout(layout_data["text_regions"], base_image_path, output_path)

    print("\n--- DEBUG INFO ---")
    print(json.dumps(layout_data, indent=2))
    if validation_errors:
        print("\n❗ Please review the above errors and consider re-prompting Maverick or adjusting constraints.")
    else:
        print("\n🎉 Layout is ready for text insertion and further design steps.")


# Saving the JSON Output

In [5]:
import json

# Save the JSON layout to a file
with open("layout.json", "w") as f:
    json.dump(layout_data, f, indent=2)
print("💾 Saved layout JSON as layout.json")


💾 Saved layout JSON as layout.json


# Adaptive Layout Architecture 


In [ ]:
import os
import base64
import json
from PIL import Image, ImageDraw
from groq import Groq

# === CONFIGURATION ===
API_KEY = "YOUR_GROQ_API_KEY_HERE"
TARGET_SIZE = 1024
layout_name = "adaptive_grid_free"
base_image_path = "/kaggle/working/edtech_1024.png"
output_path = "visualization_adaptive.png"

# === IMAGE ENCODING ===
def encode_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

base_image_b64 = encode_image_base64(base_image_path)

# === ADAPTIVE GRID-FREE PROMPT ===
adaptive_prompt = f"""
You are an EXPERT EdTech Marketing Designer creating a visually striking {TARGET_SIZE}x{TARGET_SIZE} poster for a Generative AI Bootcamp, targeting a Gen Z audience.

CRITICAL REQUIREMENTS:
- Generate EXACTLY 6 text zones with NO overlaps beyond 15px.
- Minimum 80px gap between zone edges.
- All zones must fit within the {TARGET_SIZE}x{TARGET_SIZE} canvas with 20px padding.
- Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
  - hero_headline: 600x160 max, font_size: 60px, center-aligned
  - hero_subline: 500x100 max, font_size: 40px, center-aligned
  - benefit_statement: 350x220 max, font_size: 35px
  - testimonial: 350x180 max, font_size: 32px
  - success_metrics: 400x150 max, font_size: 40px
  - cta_primary: 350x70 max, font_size: 45px
- Arrange zones dynamically, filling available space without a grid, adapting to the image's negative areas.
- Specify 'font_family' (Inter or Poppins), 'font_size', and 'text_align' in JSON output.

ZONE TYPES (must include all 6):
1. hero_headline
2. hero_subline
3. benefit_statement
4. testimonial
5. success_metrics
6. cta_primary

REQUIRED JSON FORMAT:
{{
  "visual_analysis": {{
    "poster_size": "{TARGET_SIZE}x{TARGET_SIZE}",
    "total_zones": 6,
    "layout_strategy": "{layout_name}"
  }},
  "text_regions": [
    {{
      "type": "<zone_type>",
      "x": <int>,
      "y": <int>,
      "width": <int>,
      "height": <int>,
      "font_family": "<font_name>",
      "font_size": <int>,
      "text_align": "<alignment>"
    }},
    ...
  ]
}}

Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
"""

# === LLAMA 4 MAVERICK CALL ===
def call_llama4_maverick(prompt, image_b64):
    client = Groq(api_key=API_KEY)
    print("🔄 Sending prompt and image to Llama 4 Maverick...")
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                ]
            }
        ],
        max_tokens=2048,
        temperature=0.7,
        response_format={"type": "json_object"},
    )
    print("✅ Received response from Maverick.")
    return completion.choices[0].message.content

# === VALIDATION ===
def check_adaptive_overlap(region1, region2, min_gap=80, max_overlap=15):
    x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
    x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
    x1_right, y1_bottom = x1 + w1, y1 + h1
    x2_right, y2_bottom = x2 + w2, y2 + h2
    horizontal_gap = min(abs(x1 - x2_right), abs(x1_right - x2))
    vertical_gap = min(abs(y1 - y2_bottom), abs(y1_bottom - y2))
    overlap_x = max(0, min(x1_right, x2_right) - max(x1, x2))
    overlap_y = max(0, min(y1_bottom, y2_bottom) - max(y1, y2))
    total_overlap = overlap_x * overlap_y
    if horizontal_gap < min_gap or vertical_gap < min_gap:
        return True
    if total_overlap > max_overlap:
        return True
    return False

def validate_adaptive_layout(text_regions):
    errors = []
    max_sizes = {
        "hero_headline": (600, 160, 60),
        "hero_subline": (500, 100, 40),
        "benefit_statement": (350, 220, 35),
        "testimonial": (350, 180, 32),
        "success_metrics": (400, 150, 40),
        "cta_primary": (350, 70, 45)
    }
    for i, reg1 in enumerate(text_regions):
        t = reg1["type"]
        max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
        if reg1["width"] > max_w or reg1["height"] > max_h:
            errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
        if reg1.get("font_size", 0) > max_font:
            errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
        for j, reg2 in enumerate(text_regions):
            if i < j:
                if check_adaptive_overlap(reg1, reg2):
                    errors.append(f"Overlap or insufficient gap between {reg1['type']} and {reg2['type']}")
        if reg1["x"] < 20 or reg1["y"] < 20 or reg1["x"]+reg1["width"] > TARGET_SIZE-20 or reg1["y"]+reg1["height"] > TARGET_SIZE-20:
            errors.append(f"{t} outside canvas padding")
    return errors

# === VISUALIZATION ===
def visualize_adaptive_layout(text_regions, base_image_path, filename="visualization_adaptive.png"):
    img = Image.open(base_image_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    colors = ["#FF4500", "#2E8B57", "#6A5ACD", "#FFA500", "#20B2AA", "#FF69B4"]
    for idx, reg in enumerate(text_regions):
        color = colors[idx % len(colors)]
        x, y, w, h = reg["x"], reg["y"], reg["width"], reg["height"]
        label = f"#{idx+1} {reg['type']} {reg.get('font_size','')}px"
        draw.rectangle([x, y, x+w, y+h], outline=color, width=4)
        draw.text((x+8, y+8), label, fill=color)
    img.save(filename)
    print(f"🖼️ Adaptive layout visualization saved as {filename}")

# === MAIN WORKFLOW ===
if __name__ == "__main__":
    json_str = call_llama4_maverick(adaptive_prompt, base_image_b64)
    print("🔎 Parsing JSON...")
    layout_data = json.loads(json_str)
    print(json.dumps(layout_data, indent=2))

    print("🔎 Validating layout...")
    validation_errors = validate_adaptive_layout(layout_data["text_regions"])
    if validation_errors:
        print("❗ Validation errors found:")
        for err in validation_errors:
            print("  -", err)
    else:
        print("✅ Layout passes all constraints.")

    visualize_adaptive_layout(layout_data["text_regions"], base_image_path, output_path)

    print("\n--- DEBUG INFO ---")
    print(json.dumps(layout_data, indent=2))
    if validation_errors:
        print("\n❗ Please review the above errors and consider re-prompting Maverick or adjusting constraints.")
    else:
        print("\n🎉 Adaptive layout is ready for text insertion and further design steps.")

🔄 Sending prompt and image to Llama 4 Maverick...
✅ Received response from Maverick.
🔎 Parsing JSON...
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 6,
    "layout_strategy": "adaptive_grid_free"
  },
  "text_regions": [
    {
      "type": "hero_headline",
      "x": 212,
      "y": 40,
      "width": 600,
      "height": 120,
      "font_family": "Poppins",
      "font_size": 60,
      "text_align": "center"
    },
    {
      "type": "hero_subline",
      "x": 262,
      "y": 180,
      "width": 500,
      "height": 80,
      "font_family": "Inter",
      "font_size": 40,
      "text_align": "center"
    },
    {
      "type": "benefit_statement",
      "x": 60,
      "y": 320,
      "width": 350,
      "height": 200,
      "font_family": "Inter",
      "font_size": 35,
      "text_align": "left"
    },
    {
      "type": "testimonial",
      "x": 614,
      "y": 320,
      "width": 350,
      "height": 160,
      "font_family": "Poppins",
      "font_

# Saving the JSON Output

In [9]:
import json

# Save the JSON layout to a file
with open("layout.json", "w") as f:
    json.dump(layout_data, f, indent=2)
print("💾 Saved layout JSON as layout.json")


💾 Saved layout JSON as layout.json


In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import json
import os
import qrcode

# --- CONFIGURATION ---
FONT_BASE_PATH = "/kaggle/input/fontsss"
TARGET_SIZE = 1024
CLEAN_OUTPUT_PATH = "/kaggle/working/final_poster_clean.png"
DEBUG_OUTPUT_PATH = "/kaggle/working/visualization_maverick.png"
IMAGE_PATH = "/kaggle/working/edtech_1024.png"

# Logo paths
LOGO_PATHS = {
    "linkedin": "/kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png",
    "twitter": "/kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png"
}

# Font paths for all architectures
font_paths = {
    "montserrat-bold": "/kaggle/input/data2/archive (4)/Montserrat/static/Montserrat-Bold.ttf",
    "open-sans-regular": "/kaggle/input/data2/archive (4)/Open_Sans/static/OpenSans-Regular.ttf",
    "poppins-bold": "/kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf",
    "inter-regular": "/kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf"
}

# Content mapping for each zone type
ZONE_CONTENT = {
    "hero_headline": "Master Gen AI in 12 Weeks",
    "hero_subline": "Build Tomorrow's Tech Skills Today",
    "benefit_statement": "• Build killer portfolios\n• Guaranteed job interviews",
    "testimonial": "\"This bootcamp changed my career trajectory completely!\" - Sarah K.",
    "success_metrics": "95% Job Placement\n$75K+ Average Salary",
    "cta_primary": "ENROLL NOW"
}

# --- UTILITY FUNCTIONS ---
def get_font_with_fallback(zone_type, font_size, font_family=None):
    print(f"\n🔍 Loading font for {zone_type} with size {font_size}px, family: {font_family}")
    
    # Default font mapping based on zone type
    if not font_family:
        if zone_type in ["hero_headline", "success_metrics", "cta_primary"]:
            font_family = "montserrat-bold"  # Default bold for headlines/CTAs
        else:
            font_family = "open-sans-regular"  # Default regular for others
    
    # Map font_family from JSON to available fonts
    font_map = {
        "Montserrat": "montserrat-bold",
        "Open Sans": "open-sans-regular",
        "Poppins": "poppins-bold",
        "Inter": "inter-regular"
    }
    font_key = font_map.get(font_family.split()[0], "montserrat-bold")  # Fallback to Montserrat
    
    font_path = font_paths.get(font_key)
    font_variant = font_key.replace("-", " ").title()
    
    if font_path and os.path.exists(font_path):
        try:
            print(f"   ✅ Loading {font_variant} from {font_path}")
            return ImageFont.truetype(font_path, font_size)
        except Exception as e:
            print(f"   ❌ Error loading {font_path}: {str(e)}")
    print("   ⚠️ Font loading failed, falling back to default PIL font")
    return ImageFont.load_default()

def hex_to_rgba(hex_color):
    try:
        hex_color = hex_color.lstrip("#")
        if len(hex_color) == 6:
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (255,)
        return (255, 255, 255, 255)
    except Exception as e:
        print(f"⚠️ Error converting hex color {hex_color}: {e}")
        return (255, 255, 255, 255)

def add_text_with_shadow(draw, position, text, font, text_color):
    x, y = position
    shadow_offset = 2
    draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=(0, 0, 0, 120))  # Subtle shadow
    draw.text((x, y), text, font=font, fill=text_color)  # Main text

def create_qr_code(draw, image, x, y, width, height):
    if width <= 0 or height <= 0:
        print(f"⚠️ Invalid QR dimensions: width={width}, height={height}, using defaults")
        width, height = 100, 100
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=1
    )
    qr.add_data("https://genai-bootcamp.com")
    qr.make(fit=True)
    qr_image = qr.make_image(fill_color="black", back_color="white").convert("RGBA")
    qr_image = qr_image.resize((width, height), Image.Resampling.LANCZOS)
    enhancer = ImageEnhance.Brightness(qr_image)
    qr_image = enhancer.enhance(0.8)
    image.paste(qr_image, (int(x), int(y)), qr_image)
    print(f"✅ Drew QR code at ({x}, {y}) with size {width}x{height}")

def draw_social_logos(draw, image, x, y, max_width, max_height):
    try:
        linkedin_logo = Image.open(LOGO_PATHS["linkedin"]).convert("RGBA")
        twitter_logo = Image.open(LOGO_PATHS["twitter"]).convert("RGBA")
        max_logo_size = min(40, max_height - 10)
        linkedin_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        twitter_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        linkedin_x = x
        linkedin_y = y + (max_height - linkedin_logo.height) // 2
        twitter_x = linkedin_x + linkedin_logo.width + 15
        twitter_y = y + (max_height - twitter_logo.height) // 2
        if twitter_x + twitter_logo.width > x + max_width:
            twitter_x = x + max_width - twitter_logo.width
            linkedin_x = twitter_x - linkedin_logo.width - 15
        enhancer = ImageEnhance.Brightness
        linkedin_logo = enhancer(linkedin_logo).enhance(0.7)
        twitter_logo = enhancer(twitter_logo).enhance(0.7)
        image.paste(linkedin_logo, (int(linkedin_x), int(linkedin_y)), linkedin_logo)
        image.paste(twitter_logo, (int(twitter_x), int(twitter_y)), twitter_logo)
        print(f"✅ Drew logos: LinkedIn at ({linkedin_x}, {linkedin_y}), Twitter at ({twitter_x}, {twitter_y})")
    except Exception as e:
        print(f"❌ Error drawing social logos: {e}")

def wrap_text(draw, text, font, max_width, max_height, font_size, zone_type):
    lines = []
    adjusted_font = font
    adjusted_font_size = font_size
    print(f"📏 Wrapping text for {zone_type}: '{text}'")
    print(f"    Max width: {max_width}px, Max height: {max_height}px, Initial font_size: {font_size}px")
    min_font_size = max(12, int(font_size * 0.6))
    if "•" in text:
        raw_lines = text.split("\n")
    else:
        raw_lines = [text]
    while adjusted_font_size >= min_font_size:
        current_lines = []
        for line in raw_lines:
            if not line.strip():
                continue
            if line.strip().startswith("•"):
                bullet_text = line.strip()
                text_bbox = draw.textbbox((0, 0), bullet_text, font=adjusted_font)
                text_width = text_bbox[2] - text_bbox[0]
                if text_width <= max_width - 20:
                    current_lines.append(bullet_text)
                else:
                    words = bullet_text[2:].split()
                    current_line = ["•"]
                    for word in words:
                        test_line = " ".join(current_line + [word])
                        text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                        text_width = text_bbox[2] - text_bbox[0]
                        if text_width <= max_width - 20:
                            current_line.append(word)
                        else:
                            if len(current_line) > 1:
                                current_lines.append(" ".join(current_line))
                            current_line = ["•", word]
                    if len(current_line) > 1:
                        current_lines.append(" ".join(current_line))
            else:
                words = line.split()
                current_line = []
                for word in words:
                    test_line = " ".join(current_line + [word])
                    text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                    text_width = text_bbox[2] - text_bbox[0]
                    if text_width <= max_width:
                        current_line.append(word)
                    else:
                        if current_line:
                            current_lines.append(" ".join(current_line))
                        current_line = [word]
                if current_line:
                    current_lines.append(" ".join(current_line))
        if current_lines:
            text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
            line_height = text_bbox[3] - text_bbox[1] + 6
            total_height = len(current_lines) * line_height
            print(f"    Font size {adjusted_font_size}px: {len(current_lines)} lines, Total height: {total_height}px")
            if total_height <= max_height or adjusted_font_size == min_font_size:
                lines = current_lines
                break
        adjusted_font_size -= 1
        adjusted_font = get_font_with_fallback(zone_type, adjusted_font_size)
    if not lines and text.strip():
        print("⚠️ Text doesn't fit, forcing at least one line")
        lines = [text[:30] + "..." if len(text) > 30 else text]
    print(f"    Final: {len(lines)} lines with font size {adjusted_font_size}px")
    return lines, adjusted_font, adjusted_font_size

def get_text_color_for_zone(zone_type):
    color_map = {
        "hero_headline": "#FFFFFF",
        "hero_subline": "#E0E0E0",
        "benefit_statement": "#D3D3D3",
        "testimonial": "#D3D3D3",
        "success_metrics": "#E0E0E0",
        "cta_primary": "#00CED1"
    }
    return color_map.get(zone_type, "#FFFFFF")

def get_text_alignment(zone_type, text_align):
    if zone_type in ["hero_headline", "hero_subline", "cta_primary"]:
        return "center"
    return text_align if text_align else "left"

def draw_zone_content(draw, image, zone_region, show_debug=False):
    zone_type = zone_region.get("type", "")
    x = zone_region.get("x", 0)
    y = zone_region.get("y", 0)
    width = zone_region.get("width", 300)
    height = zone_region.get("height", 100)
    font_size = zone_region.get("font_size", 24)
    text_align = zone_region.get("text_align", "left")
    font_family = zone_region.get("font_family", None)
    
    text = ZONE_CONTENT.get(zone_type, f"Sample {zone_type} text")
    text_color = hex_to_rgba(get_text_color_for_zone(zone_type))
    alignment = get_text_alignment(zone_type, text_align)
    
    print(f"\n🎨 Drawing {zone_type}: '{text}' at ({x}, {y}) size {width}x{height}")
    
    image_w, image_h = image.size
    margin = 30
    x = max(margin, min(x, image_w - width - margin))
    y = max(margin, min(y, image_h - height - margin))
    
    font = get_font_with_fallback(zone_type, font_size, font_family)
    
    lines, adjusted_font, adjusted_font_size = wrap_text(
        draw, text, font, width - 20, height - 20, font_size, zone_type
    )
    
    if not lines:
        print("⚠️ No text to draw")
        return
    
    text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
    line_height = text_bbox[3] - text_bbox[1] + 6
    total_text_height = len(lines) * line_height
    if alignment == "center":
        start_y = y + (height - total_text_height) // 2
    else:
        start_y = y + 10
    
    for i, line in enumerate(lines):
        line_y = start_y + i * line_height
        if alignment == "center":
            text_bbox = draw.textbbox((0, 0), line, font=adjusted_font)
            text_width = text_bbox[2] - text_bbox[0]
            line_x = x + (width - text_width) // 2
        else:
            line_x = x + 10
        add_text_with_shadow(draw, (line_x, line_y), line, adjusted_font, text_color)
    
    print(f"✅ Drew {zone_type} with {len(lines)} lines (font size: {adjusted_font_size}px)")
    
    if show_debug:
        debug_colors = {
            'hero_headline': '#00FFFF', 'hero_subline': '#FFFF00', 'benefit_statement': '#00FF00',
            'testimonial': '#FF00FF', 'success_metrics': '#0000FF', 'cta_primary': '#FFA500'
        }
        color = debug_colors.get(zone_type, '#FF0000')
        draw.rectangle([x, y, x + width, y + height], outline=color, width=2)
        draw.text((x + 5, y + 5), f"{zone_type}", fill=color, font=get_font_with_fallback("default", 12))

def render_poster():
    print("🎨 Llama 4 Maverick Universal Poster Renderer")
    print("=" * 50)
    
    json_path = "/kaggle/working/layout.json"
    if not os.path.exists(json_path):
        print(f"❌ JSON file not found: {json_path}")
        return None, None
    
    print(f"📄 Loading JSON from: {json_path}")
    try:
        with open(json_path, "r") as f:
            layout_data = json.load(f)
        text_regions = layout_data.get("text_regions", [])
        print(f"📊 Found {len(text_regions)} text regions")
    except Exception as e:
        print(f"❌ Error loading JSON: {e}")
        return None, None
    
    if not text_regions:
        print("❌ No text regions found in JSON!")
        return None, None
    
    print(f"🖼️ Loading base image: {IMAGE_PATH}")
    try:
        base_image = Image.open(IMAGE_PATH).convert("RGBA")
        image_w, image_h = base_image.size
        print(f"✅ Image loaded: {image_w}x{image_h}")
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None
    
    clean_image = base_image.copy()
    debug_image = base_image.copy()
    clean_draw = ImageDraw.Draw(clean_image)
    debug_draw = ImageDraw.Draw(debug_image)
    
    successful_zones = 0
    for i, region in enumerate(text_regions, 1):
        zone_type = region.get("type", f"zone_{i}")
        print(f"\n--- Processing Region {i}/{len(text_regions)}: {zone_type} ---")
        try:
            draw_zone_content(clean_draw, clean_image, region, show_debug=False)
            draw_zone_content(debug_draw, debug_image, region, show_debug=True)
            successful_zones += 1
        except Exception as e:
            print(f"❌ Error rendering {zone_type}: {e}")
            continue
    
    print("\n🎯 Adding hardcoded QR code...")
    create_qr_code(clean_draw, clean_image, 900, 880, 100, 100)
    create_qr_code(debug_draw, debug_image, 900, 880, 100, 100)
    debug_draw.rectangle([900, 880, 1000, 980], outline='#808080', width=2)
    debug_draw.text((905, 885), "QR Code", fill='#808080', font=get_font_with_fallback("default", 12))
    
    print("\n🎯 Adding hardcoded social logos...")
    draw_social_logos(clean_draw, clean_image, 30, 880, 120, 60)
    draw_social_logos(debug_draw, debug_image, 30, 880, 120, 60)
    debug_draw.rectangle([30, 880, 150, 940], outline='#4169E1', width=2)
    debug_draw.text((35, 885), "Social Logos", fill='#4169E1', font=get_font_with_fallback("default", 12))
    
    print(f"\n📊 Successfully rendered {successful_zones} text zones + QR + Logos")
    
    clean_path = None
    debug_path = None
    try:
        clean_image.save(CLEAN_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Clean poster saved: {CLEAN_OUTPUT_PATH}")
        clean_path = CLEAN_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving clean image: {e}")
    try:
        debug_image.save(DEBUG_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Debug poster saved: {DEBUG_OUTPUT_PATH}")
        debug_path = DEBUG_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving debug image: {e}")
    
    return clean_path, debug_path

if __name__ == "__main__":
    print("🚀 Starting universal poster generation from /kaggle/working/layout.json...")
    print("\n🔍 Checking required files:")
    for name, path in font_paths.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Font {name}: {path}")
    for name, path in LOGO_PATHS.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Logo {name}: {path}")
    json_path = "/kaggle/working/layout.json"
    json_exists = os.path.exists(json_path)
    status = "✅" if json_exists else "❌"
    print(f"   {status} JSON file: {json_path}")
    clean_result, debug_result = render_poster()
    if clean_result and debug_result:
        print(f"\n🎉 SUCCESS! Generated two versions:")
        print(f"   📸 CLEAN POSTER: {clean_result}")
        print(f"   🔧 DEBUG VERSION: {debug_result}")
    else:
        print("\n❌ Failed to create poster. Check error messages above.")
    print("\n🏁 Poster generation completed!")

# Horizontal Staggered Architecture


In [ ]:
# import os
# import base64
# import json
# from PIL import Image, ImageDraw
# from groq import Groq

# # === CONFIGURATION ===
# API_KEY = "YOUR_GROQ_API_KEY)HERE"
# TARGET_SIZE = 1024
# layout_name = "staggered_horizon"
# base_image_path = "/kaggle/working/edtech_1024.png"
# output_path = "visualization_staggered.png"

# # === IMAGE ENCODING ===
# def encode_image_base64(image_path):
#     with open(image_path, "rb") as img_file:
#         return base64.b64encode(img_file.read()).decode('utf-8')

# base_image_b64 = encode_image_base64(base_image_path)

# # === STAGGERED HORIZON PROMPT ===
# staggered_prompt = f"""
# You are an EXPERT EdTech Marketing Designer creating a visually striking {TARGET_SIZE}x{TARGET_SIZE} poster for a Generative AI Bootcamp, targeting a Gen Z audience.

# CRITICAL REQUIREMENTS:
# - Generate EXACTLY 7 text zones with NO overlaps beyond 10px horizontally.
# - Minimum 90px vertical gap between tiers (top, middle, bottom).
# - All zones must fit within the {TARGET_SIZE}x{TARGET_SIZE} canvas with 25px padding.
# - Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
#   - hero_headline: 500x140 max, font_size: 65px, center-aligned
#   - hero_subline: 400x80 max, font_size: 38px, center-aligned
#   - benefit_statement: 300x180 max, font_size: 32px, left-aligned
#   - testimonial: 300x150 max, font_size: 30px, right-aligned
#   - testimonial_2: 300x150 max, font_size: 30px, right-aligned
#   - cta_primary: 350x60 max, font_size: 48px, center-aligned
#   - cta_secondary: 350x60 max, font_size: 48px, center-aligned
# - Arrange zones in a staggered horizon pattern: hero_headline and hero_subline on top tier (left/right), benefit_statement, testimonial, and testimonial_2 on middle tier (left/right/center), cta_primary and cta_secondary on bottom tier (left/right).
# - Specify 'font_family' (Montserrat or Open Sans), 'font_size', and 'text_align' in JSON output.

# ZONE TYPES (must include all 7):
# 1. hero_headline
# 2. hero_subline
# 3. benefit_statement
# 4. testimonial
# 5. testimonial_2
# 6. cta_primary
# 7. cta_secondary

# REQUIRED JSON FORMAT:
# {{
#   "visual_analysis": {{
#     "poster_size": "{TARGET_SIZE}x{TARGET_SIZE}",
#     "total_zones": 7,
#     "layout_strategy": "{layout_name}"
#   }},
#   "text_regions": [
#     {{
#       "type": "<zone_type>",
#       "x": <int>,
#       "y": <int>,
#       "width": <int>,
#       "height": <int>,
#       "font_family": "<font_name>",
#       "font_size": <int>,
#       "text_align": "<alignment>"
#     }},
#     ...
#   ]
# }}

# Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
# """

# # === LLAMA 4 MAVERICK CALL ===
# def call_llama4_maverick(prompt, image_b64):
#     client = Groq(api_key=API_KEY)
#     print("🔄 Sending prompt and image to Llama 4 Maverick...")
#     completion = client.chat.completions.create(
#         model="meta-llama/llama-4-maverick-17b-128e-instruct",
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": prompt},
#                     {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
#                 ]
#             }
#         ],
#         max_tokens=2048,
#         temperature=0.7,
#         response_format={"type": "json_object"},
#     )
#     print("✅ Received response from Maverick.")
#     return completion.choices[0].message.content

# # === VALIDATION ===
# def check_staggered_overlap(region1, region2, min_vertical_gap=90, max_horizontal_overlap=10):
#     x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
#     x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
#     x1_right, y1_bottom = x1 + w1, y1 + h1
#     x2_right, y2_bottom = x2 + w2, y2 + h2
    
#     # Vertical tier check
#     vertical_gap = min(abs(y1 - y2_bottom), abs(y1_bottom - y2))
#     if vertical_gap < min_vertical_gap and region1["type"] != region2["type"]:  # Allow same tier overlap within type
#         return True
    
#     # Horizontal overlap check
#     overlap_x = max(0, min(x1_right, x2_right) - max(x1, x2))
#     if overlap_x > max_horizontal_overlap:
#         return True
    
#     return False

# def validate_staggered_layout(text_regions):
#     errors = []
#     max_sizes = {
#         "hero_headline": (500, 140, 65),
#         "hero_subline": (400, 80, 38),
#         "benefit_statement": (300, 180, 32),
#         "testimonial": (300, 150, 30),
#         "testimonial_2": (300, 150, 30),
#         "cta_primary": (350, 60, 48),
#         "cta_secondary": (350, 60, 48)
#     }
#     for i, reg1 in enumerate(text_regions):
#         t = reg1["type"]
#         max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
#         if reg1["width"] > max_w or reg1["height"] > max_h:
#             errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
#         if reg1.get("font_size", 0) > max_font:
#             errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
#         for j, reg2 in enumerate(text_regions):
#             if i < j:
#                 if check_staggered_overlap(reg1, reg2):
#                     errors.append(f"Overlap or insufficient gap between {reg1['type']} and {reg2['type']}")
#         if reg1["x"] < 25 or reg1["y"] < 25 or reg1["x"]+reg1["width"] > TARGET_SIZE-25 or reg1["y"]+reg1["height"] > TARGET_SIZE-25:
#             errors.append(f"{t} outside canvas padding")
#     return errors

# # === VISUALIZATION ===
# def visualize_staggered_layout(text_regions, base_image_path, filename="visualization_staggered.png"):
#     img = Image.open(base_image_path).convert("RGBA")
#     draw = ImageDraw.Draw(img)
#     colors = ["#FF4500", "#2E8B57", "#6A5ACD", "#FFA500", "#FF69B4", "#00CED1", "#FFD700"]  # Updated for 7 zones
#     for idx, reg in enumerate(text_regions):
#         color = colors[idx % len(colors)]
#         x, y, w, h = reg["x"], reg["y"], reg["width"], reg["height"]
#         label = f"#{idx+1} {reg['type']} {reg.get('font_size','')}px"
#         draw.rectangle([x, y, x+w, y+h], outline=color, width=4)
#         draw.text((x+8, y+8), label, fill=color)
#     img.save(filename)
#     print(f"🖼️ Staggered layout visualization saved as {filename}")

# # === MAIN WORKFLOW ===
# if __name__ == "__main__":
#     json_str = call_llama4_maverick(staggered_prompt, base_image_b64)
#     print("🔎 Parsing JSON...")
#     layout_data = json.loads(json_str)
#     print(json.dumps(layout_data, indent=2))

#     print("🔎 Validating layout...")
#     validation_errors = validate_staggered_layout(layout_data["text_regions"])
#     if validation_errors:
#         print("❗ Validation errors found:")
#         for err in validation_errors:
#             print("  -", err)
#     else:
#         print("✅ Layout passes all constraints.")

#     visualize_staggered_layout(layout_data["text_regions"], base_image_path, output_path)

#     print("\n--- DEBUG INFO ---")
#     print(json.dumps(layout_data, indent=2))
#     if validation_errors:
#         print("\n❗ Please review the above errors and consider re-prompting Maverick or adjusting constraints.")
#     else:
#         print("\n🎉 Staggered layout is ready for text insertion and further design steps.")

🔄 Sending prompt and image to Llama 4 Maverick...
✅ Received response from Maverick.
🔎 Parsing JSON...
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 7,
    "layout_strategy": "staggered_horizon"
  },
  "text_regions": [
    {
      "type": "hero_headline",
      "x": 50,
      "y": 50,
      "width": 450,
      "height": 120,
      "font_family": "Montserrat",
      "font_size": 60,
      "text_align": "left"
    },
    {
      "type": "hero_subline",
      "x": 524,
      "y": 50,
      "width": 400,
      "height": 80,
      "font_family": "Open Sans",
      "font_size": 38,
      "text_align": "right"
    },
    {
      "type": "benefit_statement",
      "x": 50,
      "y": 350,
      "width": 280,
      "height": 160,
      "font_family": "Montserrat",
      "font_size": 32,
      "text_align": "left"
    },
    {
      "type": "testimonial",
      "x": 624,
      "y": 320,
      "width": 280,
      "height": 140,
      "font_family": "Open Sans",
    

In [ ]:
import os
import base64
import json
from PIL import Image, ImageDraw
from groq import Groq

# === CONFIGURATION ===
API_KEY = "YOUR_GROQ_API_KEY_HERE"
TARGET_SIZE = 1024
layout_name = "staggered_horizon"
base_image_path = "/kaggle/working/edtech_1024.png"
output_path = "visualization_staggered.png"

# === IMAGE ENCODING ===
def encode_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

base_image_b64 = encode_image_base64(base_image_path)

# === STAGGERED HORIZON PROMPT ===
staggered_prompt = f"""
You are an EXPERT EdTech Marketing Designer creating a visually striking {TARGET_SIZE}x{TARGET_SIZE} poster for a Generative AI Bootcamp, targeting a Gen Z audience.

CRITICAL REQUIREMENTS:
- Generate EXACTLY 5 text zones with NO overlaps beyond 10px horizontally.
- Minimum 90px vertical gap between tiers (top, middle, bottom).
- All zones must fit within the {TARGET_SIZE}x{TARGET_SIZE} canvas with 25px padding.
- Use LARGER bounding boxes to prevent cramped text. For each zone, do NOT exceed these maximums:
  - hero_headline: 500x140 max, font_size: 65px, center-aligned
  - hero_subline: 400x80 max, font_size: 38px, center-aligned
  - benefit_statement: 300x180 max, font_size: 32px, left-aligned
  - testimonial: 300x150 max, font_size: 30px, right-aligned
  - cta_primary: 350x60 max, font_size: 48px, center-aligned
- Arrange zones in a staggered horizon pattern: hero_headline and hero_subline on top tier (left/right), benefit_statement and testimonial on middle tier (left/right), cta_primary on bottom tier (center).
- Specify 'font_family' (Montserrat or Open Sans), 'font_size', and 'text_align' in JSON output.

ZONE TYPES (must include all 5):
1. hero_headline
2. hero_subline
3. benefit_statement
4. testimonial
5. cta_primary

REQUIRED JSON FORMAT:
{{
  "visual_analysis": {{
    "poster_size": "{TARGET_SIZE}x{TARGET_SIZE}",
    "total_zones": 5,
    "layout_strategy": "{layout_name}"
  }},
  "text_regions": [
    {{
      "type": "<zone_type>",
      "x": <int>,
      "y": <int>,
      "width": <int>,
      "height": <int>,
      "font_family": "<font_name>",
      "font_size": <int>,
      "text_align": "<alignment>"
    }},
    ...
  ]
}}

Return ONLY the JSON object in the format above. Do NOT include any commentary, explanation, or extra text. If you violate any constraint, the output will be rejected.
"""

# === LLAMA 4 MAVERICK CALL ===
def call_llama4_maverick(prompt, image_b64):
    client = Groq(api_key=API_KEY)
    print("🔄 Sending prompt and image to Llama 4 Maverick...")
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                ]
            }
        ],
        max_tokens=2048,
        temperature=0.7,
        response_format={"type": "json_object"},
    )
    print("✅ Received response from Maverick.")
    return completion.choices[0].message.content

# === VALIDATION ===
def check_staggered_overlap(region1, region2, min_vertical_gap=90, max_horizontal_overlap=10):
    x1, y1, w1, h1 = region1["x"], region1["y"], region1["width"], region1["height"]
    x2, y2, w2, h2 = region2["x"], region2["y"], region2["width"], region2["height"]
    x1_right, y1_bottom = x1 + w1, y1 + h1
    x2_right, y2_bottom = x2 + w2, y2 + h2
    
    # Vertical tier check
    vertical_gap = min(abs(y1 - y2_bottom), abs(y1_bottom - y2))
    if vertical_gap < min_vertical_gap and region1["type"] != region2["type"]:  # Allow same tier overlap within type
        return True
    
    # Horizontal overlap check
    overlap_x = max(0, min(x1_right, x2_right) - max(x1, x2))
    if overlap_x > max_horizontal_overlap:
        return True
    
    return False

def validate_staggered_layout(text_regions):
    errors = []
    max_sizes = {
        "hero_headline": (500, 140, 65),
        "hero_subline": (400, 80, 38),
        "benefit_statement": (300, 180, 32),
        "testimonial": (300, 150, 30),
        "cta_primary": (350, 60, 48)
    }
    for i, reg1 in enumerate(text_regions):
        t = reg1["type"]
        max_w, max_h, max_font = max_sizes.get(t, (9999, 9999, 9999))
        if reg1["width"] > max_w or reg1["height"] > max_h:
            errors.append(f"{t} box exceeds max size: {reg1['width']}x{reg1['height']} (max {max_w}x{max_h})")
        if reg1.get("font_size", 0) > max_font:
            errors.append(f"{t} font size too large: {reg1.get('font_size')}px (max {max_font}px)")
        for j, reg2 in enumerate(text_regions):
            if i < j:
                if check_staggered_overlap(reg1, reg2):
                    errors.append(f"Overlap or insufficient gap between {reg1['type']} and {reg2['type']}")
        if reg1["x"] < 25 or reg1["y"] < 25 or reg1["x"]+reg1["width"] > TARGET_SIZE-25 or reg1["y"]+reg1["height"] > TARGET_SIZE-25:
            errors.append(f"{t} outside canvas padding")
    return errors

# === VISUALIZATION ===
def visualize_staggered_layout(text_regions, base_image_path, filename="visualization_staggered.png"):
    img = Image.open(base_image_path).convert("RGBA")
    draw = ImageDraw.Draw(img)
    colors = ["#FF4500", "#2E8B57", "#6A5ACD", "#FFA500", "#FF69B4"]  # Adjusted for 5 zones
    for idx, reg in enumerate(text_regions):
        color = colors[idx % len(colors)]
        x, y, w, h = reg["x"], reg["y"], reg["width"], reg["height"]
        label = f"#{idx+1} {reg['type']} {reg.get('font_size','')}px"
        draw.rectangle([x, y, x+w, y+h], outline=color, width=4)
        draw.text((x+8, y+8), label, fill=color)
    img.save(filename)
    print(f"🖼️ Staggered layout visualization saved as {filename}")

# === MAIN WORKFLOW ===
if __name__ == "__main__":
    json_str = call_llama4_maverick(staggered_prompt, base_image_b64)
    print("🔎 Parsing JSON...")
    layout_data = json.loads(json_str)
    print(json.dumps(layout_data, indent=2))

    print("🔎 Validating layout...")
    validation_errors = validate_staggered_layout(layout_data["text_regions"])
    if validation_errors:
        print("❗ Validation errors found:")
        for err in validation_errors:
            print("  -", err)
    else:
        print("✅ Layout passes all constraints.")

    visualize_staggered_layout(layout_data["text_regions"], base_image_path, output_path)

    print("\n--- DEBUG INFO ---")
    print(json.dumps(layout_data, indent=2))
    if validation_errors:
        print("\n❗ Please review the above errors and consider re-prompting Maverick or adjusting constraints.")
    else:
        print("\n🎉 Staggered layout is ready for text insertion and further design steps.")

🔄 Sending prompt and image to Llama 4 Maverick...
✅ Received response from Maverick.
🔎 Parsing JSON...
{
  "visual_analysis": {
    "poster_size": "1024x1024",
    "total_zones": 5,
    "layout_strategy": "staggered_horizon"
  },
  "text_regions": [
    {
      "type": "hero_headline",
      "x": 50,
      "y": 50,
      "width": 450,
      "height": 120,
      "font_family": "Montserrat",
      "font_size": 60,
      "text_align": "left"
    },
    {
      "type": "hero_subline",
      "x": 524,
      "y": 50,
      "width": 400,
      "height": 80,
      "font_family": "Open Sans",
      "font_size": 38,
      "text_align": "right"
    },
    {
      "type": "benefit_statement",
      "x": 50,
      "y": 290,
      "width": 300,
      "height": 180,
      "font_family": "Montserrat",
      "font_size": 32,
      "text_align": "left"
    },
    {
      "type": "testimonial",
      "x": 674,
      "y": 290,
      "width": 300,
      "height": 150,
      "font_family": "Open Sans",
    

# Saving the JSON Ouptut

In [4]:
import json

# Save the JSON layout to a file
with open("layout.json", "w") as f:
    json.dump(layout_data, f, indent=2)
print("💾 Saved layout JSON as layout.json")


💾 Saved layout JSON as layout.json


# PIL(Pillow)

# PIL(1)

In [6]:
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import json
import os
import qrcode

# --- CONFIGURATION ---
FONT_BASE_PATH = "/kaggle/input/fontsss"
TARGET_SIZE = 1024
CLEAN_OUTPUT_PATH = "/kaggle/working/final_poster_clean.png"
DEBUG_OUTPUT_PATH = "/kaggle/working/visualization_maverick.png"
IMAGE_PATH = "/kaggle/working/edtech_1024.png"

# Logo paths
LOGO_PATHS = {
    "linkedin": "/kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png",
    "twitter": "/kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png"
}

# Font paths
font_paths = {
    "poppins-bold": "/kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf",
    "inter-regular": "/kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf"
}

# Content mapping for each zone type
ZONE_CONTENT = {
    "hero_headline": "Master Gen AI in 12 Weeks",
    "hero_subline": "Build Tomorrow's Tech Skills Today",
    "benefit_statement": "• Build killer portfolios\n• Guaranteed job interviews",
    "testimonial": "\"This bootcamp changed my career trajectory completely!\" - Sarah K.",
    "success_metrics": "95% Job Placement\n$75K+ Average Salary",
    "cta_primary": "ENROLL NOW"
}

# --- UTILITY FUNCTIONS ---
def get_font_with_fallback(zone_type, font_size):
    print(f"\n🔍 Loading font for {zone_type} with size {font_size}px")
    
    # Use Poppins-Bold for headlines and CTAs, Inter-Regular for everything else
    if zone_type in ["hero_headline", "success_metrics", "cta_primary"]:
        font_path = font_paths.get("poppins-bold")
        font_variant = "Poppins-Bold"
    else:
        font_path = font_paths.get("inter-regular")
        font_variant = "Inter-Regular"
    
    if font_path and os.path.exists(font_path):
        try:
            print(f"   ✅ Loading {font_variant} from {font_path}")
            return ImageFont.truetype(font_path, font_size)
        except Exception as e:
            print(f"   ❌ Error loading {font_path}: {str(e)}")
    
    print("   ⚠️ Font loading failed, falling back to default PIL font")
    return ImageFont.load_default()

def hex_to_rgba(hex_color):
    try:
        hex_color = hex_color.lstrip("#")
        if len(hex_color) == 6:
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (255,)
        return (255, 255, 255, 255)
    except Exception as e:
        print(f"⚠️ Error converting hex color {hex_color}: {e}")
        return (255, 255, 255, 255)

def add_text_with_shadow(draw, position, text, font, text_color):
    x, y = position
    shadow_offset = 2
    draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=(0, 0, 0, 120))  # Subtle shadow
    draw.text((x, y), text, font=font, fill=text_color)  # Main text

def create_qr_code(draw, image, x, y, width, height):
    if width <= 0 or height <= 0:
        print(f"⚠️ Invalid QR dimensions: width={width}, height={height}, using defaults")
        width, height = 100, 100
    
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=1
    )
    qr.add_data("https://genai-bootcamp.com")
    qr.make(fit=True)
    
    qr_image = qr.make_image(fill_color="black", back_color="white").convert("RGBA")
    qr_image = qr_image.resize((width, height), Image.Resampling.LANCZOS)
    
    enhancer = ImageEnhance.Brightness(qr_image)
    qr_image = enhancer.enhance(0.8)
    image.paste(qr_image, (int(x), int(y)), qr_image)
    print(f"✅ Drew QR code at ({x}, {y}) with size {width}x{height}")

def draw_social_logos(draw, image, x, y, max_width, max_height):
    try:
        linkedin_logo = Image.open(LOGO_PATHS["linkedin"]).convert("RGBA")
        twitter_logo = Image.open(LOGO_PATHS["twitter"]).convert("RGBA")
        
        max_logo_size = min(40, max_height - 10)
        linkedin_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        twitter_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        
        # Position logos side by side
        linkedin_x = x
        linkedin_y = y + (max_height - linkedin_logo.height) // 2
        twitter_x = linkedin_x + linkedin_logo.width + 15
        twitter_y = y + (max_height - twitter_logo.height) // 2
        
        # Ensure logos fit within bounds
        if twitter_x + twitter_logo.width > x + max_width:
            twitter_x = x + max_width - twitter_logo.width
            linkedin_x = twitter_x - linkedin_logo.width - 15
        
        # Slightly darken logos for better integration
        enhancer = ImageEnhance.Brightness
        linkedin_logo = enhancer(linkedin_logo).enhance(0.7)
        twitter_logo = enhancer(twitter_logo).enhance(0.7)
        
        image.paste(linkedin_logo, (int(linkedin_x), int(linkedin_y)), linkedin_logo)
        image.paste(twitter_logo, (int(twitter_x), int(twitter_y)), twitter_logo)
        
        print(f"✅ Drew logos: LinkedIn at ({linkedin_x}, {linkedin_y}), Twitter at ({twitter_x}, {twitter_y})")
        
    except Exception as e:
        print(f"❌ Error drawing social logos: {e}")

def wrap_text(draw, text, font, max_width, max_height, font_size, zone_type):
    lines = []
    adjusted_font = font
    adjusted_font_size = font_size
    
    print(f"📏 Wrapping text for {zone_type}: '{text}'")
    print(f"    Max width: {max_width}px, Max height: {max_height}px, Initial font_size: {font_size}px")
    
    min_font_size = max(12, int(font_size * 0.6))
    
    # Handle bullet points
    if "•" in text:
        raw_lines = text.split("\n")
    else:
        raw_lines = [text]
    
    while adjusted_font_size >= min_font_size:
        current_lines = []
        
        for line in raw_lines:
            if not line.strip():
                continue
                
            # Handle bullet points
            if line.strip().startswith("•"):
                bullet_text = line.strip()
                # Check if bullet line fits
                text_bbox = draw.textbbox((0, 0), bullet_text, font=adjusted_font)
                text_width = text_bbox[2] - text_bbox[0]
                
                if text_width <= max_width - 20:  # 20px margin for bullet
                    current_lines.append(bullet_text)
                else:
                    # Wrap bullet text
                    words = bullet_text[2:].split()  # Remove bullet for wrapping
                    current_line = ["•"]
                    for word in words:
                        test_line = " ".join(current_line + [word])
                        text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                        text_width = text_bbox[2] - text_bbox[0]
                        if text_width <= max_width - 20:
                            current_line.append(word)
                        else:
                            if len(current_line) > 1:  # If we have more than just the bullet
                                current_lines.append(" ".join(current_line))
                            current_line = ["•", word]
                    if len(current_line) > 1:
                        current_lines.append(" ".join(current_line))
            else:
                # Regular text wrapping
                words = line.split()
                current_line = []
                for word in words:
                    test_line = " ".join(current_line + [word])
                    text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                    text_width = text_bbox[2] - text_bbox[0]
                    if text_width <= max_width:
                        current_line.append(word)
                    else:
                        if current_line:
                            current_lines.append(" ".join(current_line))
                        current_line = [word]
                if current_line:
                    current_lines.append(" ".join(current_line))
        
        # Calculate total height
        if current_lines:
            text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
            line_height = text_bbox[3] - text_bbox[1] + 6
            total_height = len(current_lines) * line_height
            
            print(f"    Font size {adjusted_font_size}px: {len(current_lines)} lines, Total height: {total_height}px")
            
            if total_height <= max_height or adjusted_font_size == min_font_size:
                lines = current_lines
                break
        
        adjusted_font_size -= 1
        adjusted_font = get_font_with_fallback(zone_type, adjusted_font_size)
    
    if not lines and text.strip():
        print("⚠️ Text doesn't fit, forcing at least one line")
        lines = [text[:30] + "..." if len(text) > 30 else text]
    
    print(f"    Final: {len(lines)} lines with font size {adjusted_font_size}px")
    return lines, adjusted_font, adjusted_font_size

def get_text_color_for_zone(zone_type):
    """Get appropriate text color for each zone type"""
    color_map = {
        "hero_headline": "#FFFFFF",
        "hero_subline": "#E0E0E0", 
        "benefit_statement": "#D3D3D3",
        "testimonial": "#D3D3D3",
        "success_metrics": "#E0E0E0",
        "cta_primary": "#00CED1"
    }
    return color_map.get(zone_type, "#FFFFFF")

def get_text_alignment(zone_type, text_align):
    """Determine text alignment based on zone type and specified alignment"""
    if zone_type in ["hero_headline", "hero_subline", "cta_primary"]:
        return "center"
    return text_align if text_align else "left"

def draw_zone_content(draw, image, zone_region, show_debug=False):
    zone_type = zone_region.get("type", "")
    x = zone_region.get("x", 0)
    y = zone_region.get("y", 0)
    width = zone_region.get("width", 300)
    height = zone_region.get("height", 100)
    font_size = zone_region.get("font_size", 24)
    text_align = zone_region.get("text_align", "left")
    
    # Get content for this zone type
    text = ZONE_CONTENT.get(zone_type, f"Sample {zone_type} text")
    text_color = hex_to_rgba(get_text_color_for_zone(zone_type))
    alignment = get_text_alignment(zone_type, text_align)
    
    print(f"\n🎨 Drawing {zone_type}: '{text}' at ({x}, {y}) size {width}x{height}")
    
    # Safety bounds check
    image_w, image_h = image.size
    margin = 30
    x = max(margin, min(x, image_w - width - margin))
    y = max(margin, min(y, image_h - height - margin))
    
    # Get font
    font = get_font_with_fallback(zone_type, font_size)
    
    # Wrap text
    lines, adjusted_font, adjusted_font_size = wrap_text(
        draw, text, font, width - 20, height - 20, font_size, zone_type
    )
    
    if not lines:
        print("⚠️ No text to draw")
        return
    
    # Calculate line height
    text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
    line_height = text_bbox[3] - text_bbox[1] + 6
    
    # Calculate starting Y position based on alignment
    total_text_height = len(lines) * line_height
    if alignment == "center":
        start_y = y + (height - total_text_height) // 2
    else:
        start_y = y + 10
    
    # Draw each line
    for i, line in enumerate(lines):
        line_y = start_y + i * line_height
        
        if alignment == "center":
            # Center align text
            text_bbox = draw.textbbox((0, 0), line, font=adjusted_font)
            text_width = text_bbox[2] - text_bbox[0]
            line_x = x + (width - text_width) // 2
        else:
            # Left align (with small margin)
            line_x = x + 10
        
        add_text_with_shadow(draw, (line_x, line_y), line, adjusted_font, text_color)
    
    print(f"✅ Drew {zone_type} with {len(lines)} lines (font size: {adjusted_font_size}px)")
    
    # Draw debug bounding box if requested
    if show_debug:
        debug_colors = {
            'hero_headline': '#00FFFF', 'hero_subline': '#FFFF00', 'benefit_statement': '#00FF00',
            'testimonial': '#FF00FF', 'success_metrics': '#0000FF', 'cta_primary': '#FFA500'
        }
        color = debug_colors.get(zone_type, '#FF0000')
        draw.rectangle([x, y, x + width, y + height], outline=color, width=2)
        # Add zone type label
        draw.text((x + 5, y + 5), f"{zone_type}", fill=color, font=get_font_with_fallback("default", 12))

def render_poster():
    """Main function to render poster from Llama 4 Maverick JSON output"""
    print("🎨 Llama 4 Maverick Poster Renderer")
    print("=" * 50)
    
    # Load JSON from file
    json_path = "/kaggle/working/layout.json"
    if not os.path.exists(json_path):
        print(f"❌ JSON file not found: {json_path}")
        return None, None
    
    print(f"📄 Loading JSON from: {json_path}")
    try:
        with open(json_path, "r") as f:
            layout_data = json.load(f)
        text_regions = layout_data.get("text_regions", [])
        print(f"📊 Found {len(text_regions)} text regions")
    except Exception as e:
        print(f"❌ Error loading JSON: {e}")
        return None, None
    
    if not text_regions:
        print("❌ No text regions found in JSON!")
        return None, None
    
    # Load base image
    print(f"🖼️ Loading base image: {IMAGE_PATH}")
    try:
        base_image = Image.open(IMAGE_PATH).convert("RGBA")
        image_w, image_h = base_image.size
        print(f"✅ Image loaded: {image_w}x{image_h}")
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None
    
    # Create clean and debug versions
    clean_image = base_image.copy()
    debug_image = base_image.copy()
    
    clean_draw = ImageDraw.Draw(clean_image)
    debug_draw = ImageDraw.Draw(debug_image)
    
    # Process each text region
    successful_zones = 0
    for i, region in enumerate(text_regions, 1):
        zone_type = region.get("type", f"zone_{i}")
        print(f"\n--- Processing Region {i}/{len(text_regions)}: {zone_type} ---")
        
        try:
            draw_zone_content(clean_draw, clean_image, region, show_debug=False)
            draw_zone_content(debug_draw, debug_image, region, show_debug=True)
            successful_zones += 1
        except Exception as e:
            print(f"❌ Error rendering {zone_type}: {e}")
            continue
    
    # Add hardcoded QR code (top-right)
    print("\n🎯 Adding hardcoded QR code...")
    create_qr_code(clean_draw, clean_image, 900, 30, 100, 100)
    create_qr_code(debug_draw, debug_image, 900, 30, 100, 100)
    # Add QR debug box
    debug_draw.rectangle([900, 30, 1000, 130], outline='#808080', width=2)
    debug_draw.text((905, 35), "QR Code", fill='#808080', font=get_font_with_fallback("default", 12))
    
    # Add hardcoded social logos (bottom-left)
    print("\n🎯 Adding hardcoded social logos...")
    draw_social_logos(clean_draw, clean_image, 30, 880, 120, 60)
    draw_social_logos(debug_draw, debug_image, 30, 880, 120, 60)
    # Add logos debug box
    debug_draw.rectangle([30, 880, 150, 940], outline='#4169E1', width=2)
    debug_draw.text((35, 885), "Social Logos", fill='#4169E1', font=get_font_with_fallback("default", 12))
    
    print(f"\n📊 Successfully rendered {successful_zones} text zones + QR + Logos")
    
    # Save images
    clean_path = None
    debug_path = None
    
    try:
        clean_image.save(CLEAN_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Clean poster saved: {CLEAN_OUTPUT_PATH}")
        clean_path = CLEAN_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving clean image: {e}")
    
    try:
        debug_image.save(DEBUG_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Debug poster saved: {DEBUG_OUTPUT_PATH}")
        debug_path = DEBUG_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving debug image: {e}")
    
    return clean_path, debug_path

# Example usage:
if __name__ == "__main__":
    print("🚀 Starting poster generation from /kaggle/working/layout.json...")
    
    # Check font and logo files
    print("\n🔍 Checking required files:")
    for name, path in font_paths.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Font {name}: {path}")
    
    for name, path in LOGO_PATHS.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Logo {name}: {path}")
    
    # Check if JSON file exists
    json_path = "/kaggle/working/layout.json"
    json_exists = os.path.exists(json_path)
    status = "✅" if json_exists else "❌"
    print(f"   {status} JSON file: {json_path}")
    
    clean_result, debug_result = render_poster()
    
    if clean_result and debug_result:
        print(f"\n🎉 SUCCESS! Generated two versions:")
        print(f"   📸 CLEAN POSTER: {clean_result}")
        print(f"   🔧 DEBUG VERSION: {debug_result}")
    else:
        print("\n❌ Failed to create poster. Check error messages above.")
    
    print("\n🏁 Poster generation completed!")

🚀 Starting poster generation from /kaggle/working/layout.json...

🔍 Checking required files:
   ✅ Font poppins-bold: /kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf
   ✅ Font inter-regular: /kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf
   ✅ Logo linkedin: /kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png
   ✅ Logo twitter: /kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png
   ✅ JSON file: /kaggle/working/layout.json
🎨 Llama 4 Maverick Poster Renderer
📄 Loading JSON from: /kaggle/working/layout.json
📊 Found 6 text regions
🖼️ Loading base image: /kaggle/working/edtech_1024.png
✅ Image loaded: 1024x1024

--- Processing Region 1/6: hero_headline ---

🎨 Drawing hero_headline: 'Master Gen AI in 12 Weeks' at (162, 124) size 680x160

🔍 Loading font for hero_headline with size 65px
   ✅ Loading Poppins-Bold from /kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf
📏 Wrapping text for hero_headline: 'Master Gen AI in 12 Weeks'
    Max wi

In [5]:
!pip install qrcode

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00


# PIL(2)

In [14]:
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import json
import os
import qrcode

# --- CONFIGURATION ---
FONT_BASE_PATH = "/kaggle/input/fontsss"
TARGET_SIZE = 1024
CLEAN_OUTPUT_PATH = "/kaggle/working/final_poster_clean.png"
DEBUG_OUTPUT_PATH = "/kaggle/working/visualization_maverick.png"
IMAGE_PATH = "/kaggle/working/edtech_1024.png"

# Logo paths
LOGO_PATHS = {
    "linkedin": "/kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png",
    "twitter": "/kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png"
}

# Font paths for all architectures
font_paths = {
    "montserrat-bold": "/kaggle/input/data2/archive (4)/Montserrat/static/Montserrat-Bold.ttf",
    "open-sans-regular": "/kaggle/input/data2/archive (4)/Open_Sans/static/OpenSans-Regular.ttf",
    "poppins-bold": "/kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf",  # Retained as fallback
    "inter-regular": "/kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf"  # Retained as fallback
}

# Content mapping for each zone type
ZONE_CONTENT = {
    "hero_headline": "Master Gen AI in 12 Weeks",
    "hero_subline": "Build Tomorrow's Tech Skills Today",
    "benefit_statement": "• Build killer portfolios\n• Guaranteed job interviews",
    "testimonial": "\"This bootcamp changed my career trajectory completely!\" - Sarah K.",
    "success_metrics": "95% Job Placement\n$75K+ Average Salary",
    "cta_primary": "ENROLL NOW"
}

# --- UTILITY FUNCTIONS ---
def get_font_with_fallback(zone_type, font_size, font_family=None):
    print(f"\n🔍 Loading font for {zone_type} with size {font_size}px, family: {font_family}")
    
    # Default font mapping based on zone type (using Montserrat and Open Sans)
    if not font_family:
        if zone_type in ["hero_headline", "success_metrics", "cta_primary"]:
            font_family = "montserrat-bold"  # Default for headlines/CTAs
        else:
            font_family = "open-sans-regular"  # Default for other zones
    
    # Map font_family from JSON to available fonts
    font_map = {
        "Montserrat": "montserrat-bold",
        "Open Sans": "open-sans-regular",
        "Poppins": "poppins-bold",
        "Inter": "inter-regular"
    }
    font_key = font_map.get(font_family.split()[0], "montserrat-bold")  # Fallback to Montserrat if unknown
    
    font_path = font_paths.get(font_key)
    font_variant = font_key.replace("-", " ").title()
    
    if font_path and os.path.exists(font_path):
        try:
            print(f"   ✅ Loading {font_variant} from {font_path}")
            return ImageFont.truetype(font_path, font_size)
        except Exception as e:
            print(f"   ❌ Error loading {font_path}: {str(e)}")
    
    print("   ⚠️ Font loading failed, falling back to default PIL font")
    return ImageFont.load_default()

def hex_to_rgba(hex_color):
    try:
        hex_color = hex_color.lstrip("#")
        if len(hex_color) == 6:
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (255,)
        return (255, 255, 255, 255)
    except Exception as e:
        print(f"⚠️ Error converting hex color {hex_color}: {e}")
        return (255, 255, 255, 255)

def add_text_with_shadow(draw, position, text, font, text_color):
    x, y = position
    shadow_offset = 2
    draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=(0, 0, 0, 120))  # Subtle shadow
    draw.text((x, y), text, font=font, fill=text_color)  # Main text

def create_qr_code(draw, image, x, y, width, height):
    if width <= 0 or height <= 0:
        print(f"⚠️ Invalid QR dimensions: width={width}, height={height}, using defaults")
        width, height = 100, 100
    
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=1
    )
    qr.add_data("https://genai-bootcamp.com")
    qr.make(fit=True)
    
    qr_image = qr.make_image(fill_color="black", back_color="white").convert("RGBA")
    qr_image = qr_image.resize((width, height), Image.Resampling.LANCZOS)
    
    enhancer = ImageEnhance.Brightness(qr_image)
    qr_image = enhancer.enhance(0.8)
    image.paste(qr_image, (int(x), int(y)), qr_image)
    print(f"✅ Drew QR code at ({x}, {y}) with size {width}x{height}")

def draw_social_logos(draw, image, x, y, max_width, max_height):
    try:
        linkedin_logo = Image.open(LOGO_PATHS["linkedin"]).convert("RGBA")
        twitter_logo = Image.open(LOGO_PATHS["twitter"]).convert("RGBA")
        
        max_logo_size = min(40, max_height - 10)
        linkedin_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        twitter_logo.thumbnail((max_logo_size, max_logo_size), Image.Resampling.LANCZOS)
        
        # Position logos side by side
        linkedin_x = x
        linkedin_y = y + (max_height - linkedin_logo.height) // 2
        twitter_x = linkedin_x + linkedin_logo.width + 15
        twitter_y = y + (max_height - twitter_logo.height) // 2
        
        # Ensure logos fit within bounds
        if twitter_x + twitter_logo.width > x + max_width:
            twitter_x = x + max_width - twitter_logo.width
            linkedin_x = twitter_x - linkedin_logo.width - 15
        
        # Slightly darken logos for better integration
        enhancer = ImageEnhance.Brightness
        linkedin_logo = enhancer(linkedin_logo).enhance(0.7)
        twitter_logo = enhancer(twitter_logo).enhance(0.7)
        
        image.paste(linkedin_logo, (int(linkedin_x), int(linkedin_y)), linkedin_logo)
        image.paste(twitter_logo, (int(twitter_x), int(twitter_y)), twitter_logo)
        
        print(f"✅ Drew logos: LinkedIn at ({linkedin_x}, {linkedin_y}), Twitter at ({twitter_x}, {twitter_y})")
        
    except Exception as e:
        print(f"❌ Error drawing social logos: {e}")

def wrap_text(draw, text, font, max_width, max_height, font_size, zone_type):
    lines = []
    adjusted_font = font
    adjusted_font_size = font_size
    
    print(f"📏 Wrapping text for {zone_type}: '{text}'")
    print(f"    Max width: {max_width}px, Max height: {max_height}px, Initial font_size: {font_size}px")
    
    min_font_size = max(12, int(font_size * 0.6))
    
    # Handle bullet points
    if "•" in text:
        raw_lines = text.split("\n")
    else:
        raw_lines = [text]
    
    while adjusted_font_size >= min_font_size:
        current_lines = []
        
        for line in raw_lines:
            if not line.strip():
                continue
                
            # Handle bullet points
            if line.strip().startswith("•"):
                bullet_text = line.strip()
                text_bbox = draw.textbbox((0, 0), bullet_text, font=adjusted_font)
                text_width = text_bbox[2] - text_bbox[0]
                
                if text_width <= max_width - 20:  # 20px margin for bullet
                    current_lines.append(bullet_text)
                else:
                    words = bullet_text[2:].split()  # Remove bullet for wrapping
                    current_line = ["•"]
                    for word in words:
                        test_line = " ".join(current_line + [word])
                        text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                        text_width = text_bbox[2] - text_bbox[0]
                        if text_width <= max_width - 20:
                            current_line.append(word)
                        else:
                            if len(current_line) > 1:
                                current_lines.append(" ".join(current_line))
                            current_line = ["•", word]
                    if len(current_line) > 1:
                        current_lines.append(" ".join(current_line))
            else:
                words = line.split()
                current_line = []
                for word in words:
                    test_line = " ".join(current_line + [word])
                    text_bbox = draw.textbbox((0, 0), test_line, font=adjusted_font)
                    text_width = text_bbox[2] - text_bbox[0]
                    if text_width <= max_width:
                        current_line.append(word)
                    else:
                        if current_line:
                            current_lines.append(" ".join(current_line))
                        current_line = [word]
                if current_line:
                    current_lines.append(" ".join(current_line))
        
        if current_lines:
            text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
            line_height = text_bbox[3] - text_bbox[1] + 6
            total_height = len(current_lines) * line_height
            
            print(f"    Font size {adjusted_font_size}px: {len(current_lines)} lines, Total height: {total_height}px")
            
            if total_height <= max_height or adjusted_font_size == min_font_size:
                lines = current_lines
                break
        
        adjusted_font_size -= 1
        adjusted_font = get_font_with_fallback(zone_type, adjusted_font_size)
    
    if not lines and text.strip():
        print("⚠️ Text doesn't fit, forcing at least one line")
        lines = [text[:30] + "..." if len(text) > 30 else text]
    
    print(f"    Final: {len(lines)} lines with font size {adjusted_font_size}px")
    return lines, adjusted_font, adjusted_font_size

def get_text_color_for_zone(zone_type):
    """Get appropriate text color for each zone type"""
    color_map = {
        "hero_headline": "#FFFFFF",
        "hero_subline": "#E0E0E0", 
        "benefit_statement": "#D3D3D3",
        "testimonial": "#D3D3D3",
        "success_metrics": "#E0E0E0",
        "cta_primary": "#00CED1"
    }
    return color_map.get(zone_type, "#FFFFFF")

def get_text_alignment(zone_type, text_align):
    """Determine text alignment based on zone type and specified alignment"""
    if zone_type in ["hero_headline", "hero_subline", "cta_primary"]:
        return "center"
    return text_align if text_align else "left"

def draw_zone_content(draw, image, zone_region, show_debug=False):
    zone_type = zone_region.get("type", "")
    x = zone_region.get("x", 0)
    y = zone_region.get("y", 0)
    width = zone_region.get("width", 300)
    height = zone_region.get("height", 100)
    font_size = zone_region.get("font_size", 24)
    text_align = zone_region.get("text_align", "left")
    font_family = zone_region.get("font_family", None)  # Get font from JSON
    
    # Get content for this zone type
    text = ZONE_CONTENT.get(zone_type, f"Sample {zone_type} text")
    text_color = hex_to_rgba(get_text_color_for_zone(zone_type))
    alignment = get_text_alignment(zone_type, text_align)
    
    print(f"\n🎨 Drawing {zone_type}: '{text}' at ({x}, {y}) size {width}x{height}")
    
    # Safety bounds check
    image_w, image_h = image.size
    margin = 30
    x = max(margin, min(x, image_w - width - margin))
    y = max(margin, min(y, image_h - height - margin))
    
    # Get font with dynamic font_family from JSON
    font = get_font_with_fallback(zone_type, font_size, font_family)
    
    # Wrap text
    lines, adjusted_font, adjusted_font_size = wrap_text(
        draw, text, font, width - 20, height - 20, font_size, zone_type
    )
    
    if not lines:
        print("⚠️ No text to draw")
        return
    
    # Calculate line height
    text_bbox = draw.textbbox((0, 0), "Ay", font=adjusted_font)
    line_height = text_bbox[3] - text_bbox[1] + 6
    
    # Calculate starting Y position based on alignment
    total_text_height = len(lines) * line_height
    if alignment == "center":
        start_y = y + (height - total_text_height) // 2
    else:
        start_y = y + 10
    
    # Draw each line
    for i, line in enumerate(lines):
        line_y = start_y + i * line_height
        
        if alignment == "center":
            text_bbox = draw.textbbox((0, 0), line, font=adjusted_font)
            text_width = text_bbox[2] - text_bbox[0]
            line_x = x + (width - text_width) // 2
        else:
            line_x = x + 10
        
        add_text_with_shadow(draw, (line_x, line_y), line, adjusted_font, text_color)
    
    print(f"✅ Drew {zone_type} with {len(lines)} lines (font size: {adjusted_font_size}px)")
    
    # Draw debug bounding box if requested
    if show_debug:
        debug_colors = {
            'hero_headline': '#00FFFF', 'hero_subline': '#FFFF00', 'benefit_statement': '#00FF00',
            'testimonial': '#FF00FF', 'success_metrics': '#0000FF', 'cta_primary': '#FFA500'
        }
        color = debug_colors.get(zone_type, '#FF0000')
        draw.rectangle([x, y, x + width, y + height], outline=color, width=2)
        draw.text((x + 5, y + 5), f"{zone_type}", fill=color, font=get_font_with_fallback("default", 12))

def render_poster():
    """Main function to render poster from Llama 4 Maverick JSON output"""
    print("🎨 Llama 4 Maverick Poster Renderer")
    print("=" * 50)
    
    # Load JSON from file
    json_path = "/kaggle/working/layout.json"
    if not os.path.exists(json_path):
        print(f"❌ JSON file not found: {json_path}")
        return None, None
    
    print(f"📄 Loading JSON from: {json_path}")
    try:
        with open(json_path, "r") as f:
            layout_data = json.load(f)
        text_regions = layout_data.get("text_regions", [])
        print(f"📊 Found {len(text_regions)} text regions")
    except Exception as e:
        print(f"❌ Error loading JSON: {e}")
        return None, None
    
    if not text_regions:
        print("❌ No text regions found in JSON!")
        return None, None
    
    # Load base image
    print(f"🖼️ Loading base image: {IMAGE_PATH}")
    try:
        base_image = Image.open(IMAGE_PATH).convert("RGBA")
        image_w, image_h = base_image.size
        print(f"✅ Image loaded: {image_w}x{image_h}")
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None
    
    # Create clean and debug versions
    clean_image = base_image.copy()
    debug_image = base_image.copy()
    
    clean_draw = ImageDraw.Draw(clean_image)
    debug_draw = ImageDraw.Draw(debug_image)
    
    # Process each text region
    successful_zones = 0
    for i, region in enumerate(text_regions, 1):
        zone_type = region.get("type", f"zone_{i}")
        print(f"\n--- Processing Region {i}/{len(text_regions)}: {zone_type} ---")
        
        try:
            draw_zone_content(clean_draw, clean_image, region, show_debug=False)
            draw_zone_content(debug_draw, debug_image, region, show_debug=True)
            successful_zones += 1
        except Exception as e:
            print(f"❌ Error rendering {zone_type}: {e}")
            continue
    
    # Add hardcoded QR code (bottom-right)
    print("\n🎯 Adding hardcoded QR code...")
    create_qr_code(clean_draw, clean_image, 900, 880, 100, 100)
    create_qr_code(debug_draw, debug_image, 900, 880, 100, 100)
    # Add QR debug box
    debug_draw.rectangle([900, 880, 1000, 980], outline='#808080', width=2)
    debug_draw.text((905, 885), "QR Code", fill='#808080', font=get_font_with_fallback("default", 12))
    
    # Add hardcoded social logos (bottom-left)
    print("\n🎯 Adding hardcoded social logos...")
    draw_social_logos(clean_draw, clean_image, 30, 880, 120, 60)
    draw_social_logos(debug_draw, debug_image, 30, 880, 120, 60)
    # Add logos debug box
    debug_draw.rectangle([30, 880, 150, 940], outline='#4169E1', width=2)
    debug_draw.text((35, 885), "Social Logos", fill='#4169E1', font=get_font_with_fallback("default", 12))
    
    print(f"\n📊 Successfully rendered {successful_zones} text zones + QR + Logos")
    
    # Save images
    clean_path = None
    debug_path = None
    
    try:
        clean_image.save(CLEAN_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Clean poster saved: {CLEAN_OUTPUT_PATH}")
        clean_path = CLEAN_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving clean image: {e}")
    
    try:
        debug_image.save(DEBUG_OUTPUT_PATH, "PNG", quality=100)
        print(f"✅ Debug poster saved: {DEBUG_OUTPUT_PATH}")
        debug_path = DEBUG_OUTPUT_PATH
    except Exception as e:
        print(f"❌ Error saving debug image: {e}")
    
    return clean_path, debug_path

# Example usage:
if __name__ == "__main__":
    print("🚀 Starting poster generation from /kaggle/working/layout.json...")
    
    # Check font and logo files
    print("\n🔍 Checking required files:")
    for name, path in font_paths.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Font {name}: {path}")
    
    for name, path in LOGO_PATHS.items():
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"   {status} Logo {name}: {path}")
    
    # Check if JSON file exists
    json_path = "/kaggle/working/layout.json"
    json_exists = os.path.exists(json_path)
    status = "✅" if json_exists else "❌"
    print(f"   {status} JSON file: {json_path}")
    
    clean_result, debug_result = render_poster()
    
    if clean_result and debug_result:
        print(f"\n🎉 SUCCESS! Generated two versions:")
        print(f"   📸 CLEAN POSTER: {clean_result}")
        print(f"   🔧 DEBUG VERSION: {debug_result}")
    else:
        print("\n❌ Failed to create poster. Check error messages above.")
    
    print("\n🏁 Poster generation completed!")

🚀 Starting poster generation from /kaggle/working/layout.json...

🔍 Checking required files:
   ✅ Font montserrat-bold: /kaggle/input/data2/archive (4)/Montserrat/static/Montserrat-Bold.ttf
   ✅ Font open-sans-regular: /kaggle/input/data2/archive (4)/Open_Sans/static/OpenSans-Regular.ttf
   ✅ Font poppins-bold: /kaggle/input/data2/archive (4)/Poppins/Poppins-Bold.ttf
   ✅ Font inter-regular: /kaggle/input/data2/archive (4)/Inter/static/Inter_18pt-Regular.ttf
   ✅ Logo linkedin: /kaggle/input/data2/archive (5)/in-logo/in-logo/LI-In-Bug.png
   ✅ Logo twitter: /kaggle/input/data2/archive (5)/large-x-logo.png.twimg.768.png
   ✅ JSON file: /kaggle/working/layout.json
🎨 Llama 4 Maverick Poster Renderer
📄 Loading JSON from: /kaggle/working/layout.json
📊 Found 6 text regions
🖼️ Loading base image: /kaggle/working/edtech_1024.png
✅ Image loaded: 1024x1024

--- Processing Region 1/6: hero_headline ---

🎨 Drawing hero_headline: 'Master Gen AI in 12 Weeks' at (162, 130) size 700x120

🔍 Loading fon

# Clearing the Kaggle Directory

In [8]:
import os
import shutil

# === CLEAR KAGGLE WORKING DIRECTORY ===
print("🧹 Starting to clear /kaggle/working/ directory...")

# Define the working directory
working_dir = "/kaggle/working/"

# Check if the directory exists
if os.path.exists(working_dir):
    try:
        # Iterate over all items in the directory
        for item in os.listdir(working_dir):
            item_path = os.path.join(working_dir, item)
            if os.path.isfile(item_path):
                os.remove(item_path)  # Remove files
                print(f"🗑️ Removed file: {item}")
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)  # Remove directories and their contents
                print(f"🗑️ Removed directory: {item}")
        print("✅ /kaggle/working/ directory cleared successfully!")
    except Exception as e:
        print(f"❗ Error clearing directory: {e}")
else:
    print("⚠️ /kaggle/working/ directory does not exist.")

🧹 Starting to clear /kaggle/working/ directory...
🗑️ Removed file: final_poster_clean.png
🗑️ Removed directory: cache
🗑️ Removed directory: .virtual_documents
🗑️ Removed file: visualization_maverick.png
🗑️ Removed file: visualization_chosen.png
🗑️ Removed file: layout.json
🗑️ Removed file: edtech_1024.png
✅ /kaggle/working/ directory cleared successfully!
